# NearestNeighbor1234

This analysis compares four measured target (e.g. FGFR1/2/3/4) localization distributions between not active and active cells by using nearest neighbor distances:
- Calculate the nearest neighbor distances
- Calculate the percentage of nearest neighbor distances less or equal to a threshold 
- Determine the nearest neighbor types <br/>

The analysis is executed for each measurement and also displayed as an average for the measurement conditions not active / active.

## Load imports and dataset

In [ ]:
import numpy as np
import plotly
import plotly.graph_objects as go
import plotly.figure_factory as ff
import plotly.express as px
import plotly.io as pio
from plotly.subplots import make_subplots
import math
import h5py
from tqdm.notebook import tnrange, tqdm
import time
from textwrap import wrap

- Each measurement consists of four hdf5 files (FGFR1/2/3/4), created after the DBSCAN in Picasso (https://picassosr.readthedocs.io/en/latest/index.html). The files have to be added at the same index to the respective lists.  

In [ ]:
# store all not active measurements
not_active_FGFR1_paths, not_active_FGFR2_paths, not_active_FGFR3_paths, not_active_FGFR4_paths = [], [], [], []
# measurement 1, repeat following 8 lines for additional measurements
m1_not_active_path_FGFR1 = r"C:\path_to_target1\file1.hdf5"
m1_not_active_path_FGFR2 = r"C:\path_to_target2\file2.hdf5"
m1_not_active_path_FGFR3 = r"C:\path_to_target3\file3.hdf5"
m1_not_active_path_FGFR4 = r"C:\path_to_target4\file4.hdf5"
not_active_FGFR1_paths.append(m1_not_active_path_FGFR1)
not_active_FGFR2_paths.append(m1_not_active_path_FGFR2)
not_active_FGFR3_paths.append(m1_not_active_path_FGFR3)
not_active_FGFR4_paths.append(m1_not_active_path_FGFR4)
# measurement 2
m2_not_active_path_FGFR1 = r"C:\path_to_target1\file1.hdf5"
m2_not_active_path_FGFR2 = r"C:\path_to_target2\file2.hdf5"
m2_not_active_path_FGFR3 = r"C:\path_to_target3\file3.hdf5"
m2_not_active_path_FGFR4 = r"C:\path_to_target4\file4.hdf5"
not_active_FGFR1_paths.append(m2_not_active_path_FGFR1)
not_active_FGFR2_paths.append(m2_not_active_path_FGFR2)
not_active_FGFR3_paths.append(m2_not_active_path_FGFR3)
not_active_FGFR4_paths.append(m2_not_active_path_FGFR4)

# store all active measurements
active_FGFR1_paths, active_FGFR2_paths, active_FGFR3_paths, active_FGFR4_paths = [], [], [], []
# measurement 1
m1_active_path_FGFR1 = r"C:\path_to_target1\file1.hdf5"
m1_active_path_FGFR2 = r"C:\path_to_target2\file2.hdf5"
m1_active_path_FGFR3 = r"C:\path_to_target3\file3.hdf5"
m1_active_path_FGFR4 = r"C:\path_to_target4\file4.hdf5"
active_FGFR1_paths.append(m1_active_path_FGFR1)
active_FGFR2_paths.append(m1_active_path_FGFR2)
active_FGFR3_paths.append(m1_active_path_FGFR3)
active_FGFR4_paths.append(m1_active_path_FGFR4)

In [ ]:
# Read the hdf5-files
def read_hdf5_files(FGFR_path):
    FGFR_hdf5s = []
    for fgfr in FGFR_path:
        h5_fgfr = h5py.File(fgfr, "r")
        FGFR_hdf5s.append(h5_fgfr)
    return FGFR_hdf5s

FGFR1_active, FGFR2_active, FGFR3_active, FGFR4_active = read_hdf5_files(active_FGFR1_paths), read_hdf5_files(active_FGFR2_paths), read_hdf5_files(active_FGFR3_paths), read_hdf5_files(active_FGFR4_paths)
FGFR1_not_active, FGFR2_not_active, FGFR3_not_active, FGFR4_not_active = read_hdf5_files(not_active_FGFR1_paths), read_hdf5_files(not_active_FGFR2_paths), read_hdf5_files(not_active_FGFR3_paths), read_hdf5_files(not_active_FGFR4_paths)

## Measurement and plot settings

- Define the names of the four measured targets as strings in a list.
- Define two measurement conditions.
- Insert pixel size in nm & the amount of pixels of the detector in a row.

In [ ]:
target_names = ["FGFR1", "FGFR2", "FGFR3", "FGFR4"]  # has to be the same order as the file paths per measurement
condition_names = ["not active", "active"]  # condition names, same as the loading order in "load data"
pixel_size = 158  # in nm, important parameter, necessary for further analysis!
n_pixels = 256

- Choose a qualitative color palette or define four colors to be used for the four targets.
    - A color palette is chosen by setting the target_colors variable to px.colors.qualitative.name, e.g. target_colors  = px.colors.qualitative.Bold
    - Individual colors can be defined as a list, valid list entries are CSS-color list names, hex-code and rgb, e.g. target_colors  = ["Teal", "#636EFA'", "rgb(245, 2, 49)", "Sienna"]

In [ ]:
# possible color palettes
fig = px.colors.qualitative.swatches()
fig.show()

In [ ]:
# choose four colors that represent the targets
target_colors = px.colors.qualitative.Bold

- Choose two colors that represent not active and active measurements.

In [ ]:
condition_colors = ["RoyalBlue", "Red"]  # first = not active, second = active

# display colors
fig = go.Figure()
fig.add_trace(go.Bar(y=["Conditions "], x=[1], name="not active", orientation="h", marker=dict(color=condition_colors[0])))
fig.add_trace(go.Bar(y=["Conditions "], x=[1], name="active", orientation="h", marker=dict(color=condition_colors[1])))
fig.update_layout(barmode="stack", showlegend=False, height=200, width=400, )
fig.update_xaxes(showticklabels=False)
fig.show()

In [ ]:
# choose background color of plots: white, gray, black
background_template = "plotly_white"  # "plotly", "plotly_white" or "plotly_dark"

## Number of targets per measurement

- Display the number of target localizations.
- Optionally insert the cell areas of the measurements into the lists (e.g. cell_sizes_active = [150.6, 89] -> m1_cell_size=150.6, m2_cell_size=89 ...), counts/area will be shown instead.

In [ ]:
# the lists are either empty or contain the cell sizes per measurement
cell_sizes_active = []
cell_sizes_nactive = []

In [ ]:
fig = make_subplots(rows=1, cols=2, subplot_titles=(condition_names[0], condition_names[1]), specs=[[{"type": "bar"}, {"type": "bar"}]])

measurements_nactive = ["measurement " + str(i+1) for i in range(len(FGFR1_not_active))]
measurements_active = ["measurement " + str(i+1) for i in range(len(FGFR1_active))]

# plot the number or the number per area depending on whether the area is given
if cell_sizes_active and cell_sizes_nactive:
    number_fgfr1_active = [len(i["clusters"]["com_x"])/size for i, size in zip(FGFR1_active, cell_sizes_active)]
    number_fgfr2_active = [len(i["clusters"]["com_x"])/size for i, size in zip(FGFR2_active, cell_sizes_active)]
    number_fgfr3_active = [len(i["clusters"]["com_x"])/size for i, size in zip(FGFR3_active, cell_sizes_active)]
    number_fgfr4_active = [len(i["clusters"]["com_x"])/size for i, size in zip(FGFR4_active, cell_sizes_active)]
    number_fgfr1_nactive = [len(i["clusters"]["com_x"])/size for i, size in zip(FGFR1_not_active, cell_sizes_nactive)]
    number_fgfr2_nactive = [len(i["clusters"]["com_x"])/size for i, size in zip(FGFR2_not_active, cell_sizes_nactive)]
    number_fgfr3_nactive = [len(i["clusters"]["com_x"])/size for i, size in zip(FGFR3_not_active, cell_sizes_nactive)]
    number_fgfr4_nactive = [len(i["clusters"]["com_x"])/size for i, size in zip(FGFR4_not_active, cell_sizes_nactive)]
    y_axis_title = "counts/area" #\u00B5m²
else:
    number_fgfr1_active = [len(i["clusters"]["com_x"]) for i in FGFR1_active]
    number_fgfr2_active = [len(i["clusters"]["com_x"]) for i in FGFR2_active]
    number_fgfr3_active = [len(i["clusters"]["com_x"]) for i in FGFR3_active]
    number_fgfr4_active = [len(i["clusters"]["com_x"]) for i in FGFR4_active]
    number_fgfr1_nactive = [len(i["clusters"]["com_x"]) for i in FGFR1_not_active]
    number_fgfr2_nactive = [len(i["clusters"]["com_x"]) for i in FGFR2_not_active]
    number_fgfr3_nactive = [len(i["clusters"]["com_x"]) for i in FGFR3_not_active]
    number_fgfr4_nactive = [len(i["clusters"]["com_x"]) for i in FGFR4_not_active]
    y_axis_title = "counts"

def plot_distances_below_mean(FGFR_distances_nactive_mean, FGFR_distances_active_mean,
                             FGFR_distances_nactive_mean_error, FGFR_distances_active_mean_error,
                             y_axis_max=None):
    x_desc = target_names
    fig = go.Figure(data=[go.Bar(name=condition_names[0], x=x_desc, y=FGFR_distances_nactive_mean,
                                 marker=dict(color=condition_colors[0]),
                                error_y=dict(type="data", array=FGFR_distances_nactive_mean_error, visible=True)),
                         go.Bar(name=condition_names[1], x=x_desc, y=FGFR_distances_active_mean,
                                marker=dict(color=condition_colors[1]),
                                error_y=dict(type="data", array=FGFR_distances_active_mean_error, visible=True))])  
    fig.update_layout(title_text="Mean number of targets",
                     barmode="group", template=background_template)
    fig.update_yaxes(title_text="counts", range=[0,y_axis_max])
    fig.show()

def calc_mean_error(measurements):
    if len(measurements) > 1:
        measurements_mean = np.mean(measurements, axis=0)
        measurements_mean_error = np.std(measurements, axis=0, ddof=1)/math.sqrt(len(measurements))
    else:
        measurements_mean = measurements[0]
        measurements_mean_error = [0]*len(measurements[0])
    return measurements_mean, measurements_mean_error

number_FGFR1_nactive_mean, number_FGFR1_nactive_mean_error = calc_mean_error(number_fgfr1_nactive)
number_FGFR2_nactive_mean, number_FGFR2_nactive_mean_error = calc_mean_error(number_fgfr2_nactive)
number_FGFR3_nactive_mean, number_FGFR3_nactive_mean_error = calc_mean_error(number_fgfr3_nactive)
number_FGFR4_nactive_mean, number_FGFR4_nactive_mean_error = calc_mean_error(number_fgfr4_nactive)
number_FGFR1_active_mean, number_FGFR1_active_mean_error = calc_mean_error(number_fgfr1_active)
number_FGFR2_active_mean, number_FGFR2_active_mean_error = calc_mean_error(number_fgfr2_active)
number_FGFR3_active_mean, number_FGFR3_active_mean_error = calc_mean_error(number_fgfr3_active)
number_FGFR4_active_mean, number_FGFR4_active_mean_error = calc_mean_error(number_fgfr4_active)
FGFR_nactive_mean = [number_FGFR1_nactive_mean, number_FGFR2_nactive_mean, number_FGFR3_nactive_mean, number_FGFR4_nactive_mean]
FGFR_active_mean = [number_FGFR1_active_mean, number_FGFR2_active_mean, number_FGFR3_active_mean, number_FGFR4_active_mean]
FGFR_nactive_mean_error = [number_FGFR1_nactive_mean_error, number_FGFR2_nactive_mean_error, number_FGFR3_nactive_mean_error, number_FGFR4_nactive_mean_error]
FGFR_active_mean_error = [number_FGFR1_active_mean_error, number_FGFR2_active_mean_error, number_FGFR3_active_mean_error, number_FGFR4_active_mean_error]
plot_distances_below_mean(FGFR_nactive_mean, FGFR_active_mean, FGFR_nactive_mean_error, FGFR_active_mean_error)

# display the number of targets per single measurement and condition
fig.add_trace(go.Bar(x=measurements_nactive, y=number_fgfr1_nactive, name=target_names[0]+" "+condition_names[0], marker=dict(color=target_colors[0])), row=1, col=1)
fig.add_trace(go.Bar(x=measurements_nactive, y=number_fgfr2_nactive, name=target_names[1]+" "+condition_names[0], marker=dict(color=target_colors[1])), row=1, col=1)
fig.add_trace(go.Bar(x=measurements_nactive, y=number_fgfr3_nactive, name=target_names[2]+" "+condition_names[0], marker=dict(color=target_colors[2])), row=1, col=1)
fig.add_trace(go.Bar(x=measurements_nactive, y=number_fgfr4_nactive, name=target_names[3]+" "+condition_names[0], marker=dict(color=target_colors[3])), row=1, col=1)
  
fig.add_trace(go.Bar(x=measurements_active, y=number_fgfr1_active, name=target_names[0]+" "+condition_names[1], marker=dict(color=target_colors[0])), row=1, col=2)
fig.add_trace(go.Bar(x=measurements_active, y=number_fgfr2_active, name=target_names[1]+" "+condition_names[1], marker=dict(color=target_colors[1])), row=1, col=2)
fig.add_trace(go.Bar(x=measurements_active, y=number_fgfr3_active, name=target_names[2]+" "+condition_names[1], marker=dict(color=target_colors[2])), row=1, col=2)
fig.add_trace(go.Bar(x=measurements_active, y=number_fgfr4_active, name=target_names[3]+" "+condition_names[1], marker=dict(color=target_colors[3])), row=1, col=2)
              
fig.update_layout(title_text="Number of targets per measurement",
                 barmode="group", template=background_template)
fig.update_yaxes(title_text=y_axis_title, row=1, col=1)
fig.update_yaxes(title_text=y_axis_title, row=1, col=2)

fig.show()

## Nearest neighbor distances with one neighbor type

### Nearest neighbor distances

Calculate the nearest neighbor distances (X->Y, X=center, Y=neighbor). The neighbor space contains only one target type per analysis, the other targets are ignored.
- FGFR 1->1, 1->2, 1->3, 1->4 
- FGFR 2->1, 2->2, 2->3, 2->4
- FGFR 3->1, 3->2, 3->3, 3->4
- FGFR 4->1, 4->2, 4->3, 4->4

#### Execute the analysis

In [ ]:
def xy_stack(x, y, pixel_size):
    """
    Calculate the xy-coordinates in nm and stack them together.
    
    :param x: x coordinates [px].
    :param y: y coordinates [px].
    :param pixel_size: Size of the pixel in nm.
    :return: xy-coordinates with shape (len, 2).
    :rtype: Numpy ndarray.
    """
    x, y = x*pixel_size, y*pixel_size
    xy_stack = np.column_stack((x, y))
    return xy_stack

class GridNNSearch:
    """
    The computing time of the nearest neighbor analysis is accelerated by a grid, which lies over the space
    of the localizations, thereby creating subregions. Only neighbours in adjacent subregions are accepted as
    potential nearest neighbor candidates for a center.
    """
    def __init__(self, centers, neighbors, number_subregions, number_pixels, pixel_size):
        self.number_pixels = number_pixels  # 256
        self.pixel_size = pixel_size  # 158 nm
        self.number_subregions = number_subregions  # e.g. ~ number of neighbor points
        self.border_length = number_pixels*pixel_size/number_subregions  # length of subregion border
        self.neighbors = neighbors  # list of list of neighbors xy-locations
        self.centers = centers  # xy-locations of centers
        self.grid_neighbors = self.create_grid(number_subregions)  # subregions linked to their points (xy-locs)
        self.center_is_neighbor = self.center_is_neighbor()  # list of booleans if center list equals a neighbor list

    def center_is_neighbor(self):
        """
        Check if the center list is equal with each of the neighbor lists.
        :return: List of booleans if center list equals neighbor lists
        """
        center_is_neighbor = []
        for neighbor_list in self.neighbors:
            is_same = True if np.array_equal(self.centers, neighbor_list) else False
            center_is_neighbor.append(is_same)
        return center_is_neighbor

    def create_grid(self, number_subregions):
        """
        Divide the space into subregions that store the localizations of neighbors in their subspace.
        :param number_subregions: Total number of subregions.
        :return: Dict of key = subregion index & value = list of neighbors, their original indices and reference
        to neighbor list.
        """
        # dict with key = XY and value = []
        grid_neighbors = {}
        for x in range(number_subregions):
            for y in range(number_subregions):
                grid_neighbors.update({(x,y): []})
        # store neighbor xy-locations in corresponding subregion
        for c, neighbor_list in enumerate(self.neighbors):
            for i, neighbor in enumerate(neighbor_list):
                subregion = self.point_to_subregion(neighbor)
                # i = idx in list, c = idx of neighbor list (FGFR1 -> 0, FGFR2 -> 1 ...)
                grid_neighbors[subregion].append((neighbor, i, c))
        return grid_neighbors

    def get_nn_distances(self):
        """
        Calculate the distance to the nearest neighbor and get its type for each center.
        :return: Nearest neighbor distances and types.
        """
        nn_distances = []
        nn_types = []
        for center_idx, center in enumerate(self.centers):
            subregion = self.point_to_subregion(center)
            search_area = 0
            found_neighbor = False
            while not found_neighbor:
                # check if neighbors are in search_area range & if the neighbor is not the particle itself
                # if no neighbors are found, increase the search_area
                # for each valid sub region get the xy-localizations
                neighbors = list(map(lambda x: self.grid_neighbors[x], self.get_valid_sub_regions(subregion, search_area)))
                # merge elements of sublists to one list
                neighbors_conc = [j for i in neighbors for j in i]
                # delete the element in neighbors that equals center
                for c, neighbor in enumerate(neighbors_conc):
                    if np.array_equal(neighbor[0], center) and neighbor[1] == center_idx:
                        neighbors_conc.pop(c)
                # set appropriate search area (area that found neighbor +1)
                if neighbors_conc:
                    search_area += 1
                    found_neighbor = True
                else:
                    search_area += 1
            valid_sub_regions = self.get_valid_sub_regions(subregion, search_area)
            # get all potential nearest neighbor candidates from the valid subregions
            neighbors_in_area = list(map(lambda x: self.grid_neighbors[x], valid_sub_regions))
            neighbors_in_area_conc = [j for i in neighbors_in_area for j in i]
            for c, neighbor in enumerate(neighbors_in_area_conc):
                if np.array_equal(neighbor[0], center) and neighbor[1] == center_idx:
                    neighbors_in_area_conc.pop(c)
            # calc nearest neighbor distance and type (type refers to the index of lists of self.neighbors)
            min_distance, nn_type = self.calc_min_distance(center, neighbors_in_area_conc)
            nn_distances.append(min_distance)
            nn_types.append(nn_type)
        return nn_distances, nn_types

    def calc_min_distance(self, center, neighbor_candidates):
        """
        Calculate the euclidean distance of candidates in a list and return the nearest neighbor distance and type.
        :param center: Target center.
        :param neighbor_candidates: Potential nearest neighbors.
        :return: Nearest neighbor distance and type (type = index of list in self.neighbors).
        """
        min_distance = math.inf
        nn_type = ""
        for idx in range(len(neighbor_candidates)):
            distance = np.linalg.norm(center-neighbor_candidates[idx][0])
            if distance < min_distance:
                min_distance = distance
                nn_type = neighbor_candidates[idx][2]
        return min_distance, nn_type

    def point_to_subregion(self, point):
        """
        Sort a xy-localization to a subregion.
        :param point: xy-localization.
        :return: (x, y) refers to the subregion indices.
        """
        x = int(np.floor(point[0] / self.border_length))
        y = int(np.floor(point[1] / self.border_length))
        # this only happens if the localization is directly at the border of the measurement space.
        if x == self.number_subregions:
            x -= 1
        if y == self.number_subregions:
            y -= 1
        return x, y

    def get_valid_sub_regions(self, center, search_area):
        """
        From a center subregion get all valid subregions within the appropriate search area.
        :param center: Subregion indices.
        :param search_area: Valid deviations of center subregion-id.
        :return: List of valid subregion-ids.
        """
        valid_sub_regions = []
        for x in range(center[0] - search_area, center[0] + search_area + 1):
            for y in range(center[1] - search_area, center[1] + search_area + 1):
                if x in range(self.number_subregions) and y in range(self.number_subregions):
                    valid_sub_regions.append((x, y))
        return valid_sub_regions
    
    
start = time.time()
reduce_size = None  # testing purpose
print("Analyzing not active measurements ...")
FGFR1_nn_distances_nactive = [] # stores distances FGFR1->1/2/3/4 of all measurement
FGFR2_nn_distances_nactive = []
FGFR3_nn_distances_nactive = []
FGFR4_nn_distances_nactive = []
with tqdm(total=len(FGFR1_not_active), desc="Measurement") as pbar:
    for fgfr1, fgfr2, fgfr3, fgfr4 in zip(FGFR1_not_active, FGFR2_not_active, FGFR3_not_active, FGFR4_not_active):
        with tqdm(total=4, leave=False, desc="FGFR") as pbar_B:
            # localization unit px -> nm
            FGFR1_nactive_xy = xy_stack(fgfr1["clusters"]["com_x"][:reduce_size], fgfr1["clusters"]["com_y"][:reduce_size], pixel_size=pixel_size)
            FGFR2_nactive_xy = xy_stack(fgfr2["clusters"]["com_x"][:reduce_size], fgfr2["clusters"]["com_y"][:reduce_size], pixel_size=pixel_size)
            FGFR3_nactive_xy = xy_stack(fgfr3["clusters"]["com_x"][:reduce_size], fgfr3["clusters"]["com_y"][:reduce_size], pixel_size=pixel_size)
            FGFR4_nactive_xy = xy_stack(fgfr4["clusters"]["com_x"][:reduce_size], fgfr4["clusters"]["com_y"][:reduce_size], pixel_size=pixel_size)
            # store the neighbor xy-localizations of one measurement in lists
            neighbors_nactive_FGFR1, neighbors_nactive_FGFR2, neighbors_nactive_FGFR3, neighbors_nactive_FGFR4 = [], [], [], []
            neighbors_nactive_FGFR1.append(FGFR1_nactive_xy[:reduce_size])
            neighbors_nactive_FGFR2.append(FGFR2_nactive_xy[:reduce_size])
            neighbors_nactive_FGFR3.append(FGFR3_nactive_xy[:reduce_size])
            neighbors_nactive_FGFR4.append(FGFR4_nactive_xy[:reduce_size])
            # stores distances FGFR1->1/2/3/4 of one measurement
            FGFR1_distances = []  
            grid_11 = GridNNSearch(FGFR1_nactive_xy[:reduce_size], neighbors_nactive_FGFR1,
                                   int(np.floor(math.sqrt(len(neighbors_nactive_FGFR1[0])))), n_pixels, pixel_size)
            nn_11, _ = grid_11.get_nn_distances()
            grid_12 = GridNNSearch(FGFR1_nactive_xy[:reduce_size], neighbors_nactive_FGFR2,
                                   int(np.floor(math.sqrt(len(neighbors_nactive_FGFR2[0])))), n_pixels, pixel_size)
            nn_12, _ = grid_12.get_nn_distances()
            grid_13 = GridNNSearch(FGFR1_nactive_xy[:reduce_size], neighbors_nactive_FGFR3,
                                   int(np.floor(math.sqrt(len(neighbors_nactive_FGFR3[0])))), n_pixels, pixel_size)
            nn_13, _ = grid_13.get_nn_distances()
            grid_14 = GridNNSearch(FGFR1_nactive_xy[:reduce_size], neighbors_nactive_FGFR4,
                                   int(np.floor(math.sqrt(len(neighbors_nactive_FGFR4[0])))), n_pixels, pixel_size)
            nn_14, _ = grid_14.get_nn_distances()
            FGFR1_distances.append(nn_11); FGFR1_distances.append(nn_12)
            FGFR1_distances.append(nn_13); FGFR1_distances.append(nn_14)
            FGFR1_nn_distances_nactive.append(FGFR1_distances)
            pbar_B.update(1)
            FGFR2_distances = []
            grid_21 = GridNNSearch(FGFR2_nactive_xy[:reduce_size], neighbors_nactive_FGFR1,
                                   int(np.floor(math.sqrt(len(neighbors_nactive_FGFR1[0])))), n_pixels, pixel_size)
            nn_21, _ = grid_21.get_nn_distances()
            grid_22 = GridNNSearch(FGFR2_nactive_xy[:reduce_size], neighbors_nactive_FGFR2,
                                   int(np.floor(math.sqrt(len(neighbors_nactive_FGFR2[0])))), n_pixels, pixel_size)
            nn_22, _ = grid_22.get_nn_distances()
            grid_23 = GridNNSearch(FGFR2_nactive_xy[:reduce_size], neighbors_nactive_FGFR3,
                                   int(np.floor(math.sqrt(len(neighbors_nactive_FGFR3[0])))), n_pixels, pixel_size)
            nn_23, _ = grid_23.get_nn_distances()
            grid_24 = GridNNSearch(FGFR2_nactive_xy[:reduce_size], neighbors_nactive_FGFR4,
                                   int(np.floor(math.sqrt(len(neighbors_nactive_FGFR4[0])))), n_pixels, pixel_size)
            nn_24, _ = grid_24.get_nn_distances()
            FGFR2_distances.append(nn_21); FGFR2_distances.append(nn_22)
            FGFR2_distances.append(nn_23); FGFR2_distances.append(nn_24)
            FGFR2_nn_distances_nactive.append(FGFR2_distances)
            pbar_B.update(1)
            FGFR3_distances = []
            grid_31 = GridNNSearch(FGFR3_nactive_xy[:reduce_size], neighbors_nactive_FGFR1,
                                   int(np.floor(math.sqrt(len(neighbors_nactive_FGFR1[0])))), n_pixels, pixel_size)
            nn_31, _ = grid_31.get_nn_distances()
            grid_32 = GridNNSearch(FGFR3_nactive_xy[:reduce_size], neighbors_nactive_FGFR2,
                                   int(np.floor(math.sqrt(len(neighbors_nactive_FGFR2[0])))), n_pixels, pixel_size)
            nn_32, _ = grid_32.get_nn_distances()
            grid_33 = GridNNSearch(FGFR3_nactive_xy[:reduce_size], neighbors_nactive_FGFR3,
                                   int(np.floor(math.sqrt(len(neighbors_nactive_FGFR3[0])))), n_pixels, pixel_size)
            nn_33, _ = grid_33.get_nn_distances()
            grid_34 = GridNNSearch(FGFR3_nactive_xy[:reduce_size], neighbors_nactive_FGFR4,
                                   int(np.floor(math.sqrt(len(neighbors_nactive_FGFR4[0])))), n_pixels, pixel_size)
            nn_34, _ = grid_34.get_nn_distances()
            FGFR3_distances.append(nn_31); FGFR3_distances.append(nn_32)
            FGFR3_distances.append(nn_33); FGFR3_distances.append(nn_34)
            FGFR3_nn_distances_nactive.append(FGFR3_distances)
            pbar_B.update(1)
            FGFR4_distances = []
            grid_41 = GridNNSearch(FGFR4_nactive_xy[:reduce_size], neighbors_nactive_FGFR1,
                                   int(np.floor(math.sqrt(len(neighbors_nactive_FGFR1[0])))), n_pixels, pixel_size)
            nn_41, _ = grid_41.get_nn_distances()
            grid_42 = GridNNSearch(FGFR4_nactive_xy[:reduce_size], neighbors_nactive_FGFR2,
                                   int(np.floor(math.sqrt(len(neighbors_nactive_FGFR2[0])))), n_pixels, pixel_size)
            nn_42, _ = grid_42.get_nn_distances()
            grid_43 = GridNNSearch(FGFR4_nactive_xy[:reduce_size], neighbors_nactive_FGFR3,
                                   int(np.floor(math.sqrt(len(neighbors_nactive_FGFR3[0])))), n_pixels, pixel_size)
            nn_43, _ = grid_43.get_nn_distances()
            grid_44 = GridNNSearch(FGFR4_nactive_xy[:reduce_size], neighbors_nactive_FGFR4,
                                   int(np.floor(math.sqrt(len(neighbors_nactive_FGFR4[0])))), n_pixels, pixel_size)
            nn_44, _ = grid_44.get_nn_distances()
            FGFR4_distances.append(nn_41); FGFR4_distances.append(nn_42)
            FGFR4_distances.append(nn_43); FGFR4_distances.append(nn_44)
            FGFR4_nn_distances_nactive.append(FGFR4_distances)
            pbar_B.update(1)
            pbar.update(1)
        
print("Analyzing active measurements ...")
FGFR1_nn_distances_active = [] # stores distances FGFR1->1/2/3/4 of all measurement
FGFR2_nn_distances_active = []
FGFR3_nn_distances_active = []
FGFR4_nn_distances_active = []
with tqdm(total=len(FGFR1_active), desc="Measurement") as pbar:
    for fgfr1, fgfr2, fgfr3, fgfr4 in zip(FGFR1_active, FGFR2_active, FGFR3_active, FGFR4_active):
        with tqdm(total=4, desc="FGFR", leave=False) as pbar_B:
            # localization unit px -> nm
            FGFR1_active_xy = xy_stack(fgfr1["clusters"]["com_x"][:reduce_size], fgfr1["clusters"]["com_y"][:reduce_size], pixel_size=pixel_size)
            FGFR2_active_xy = xy_stack(fgfr2["clusters"]["com_x"][:reduce_size], fgfr2["clusters"]["com_y"][:reduce_size], pixel_size=pixel_size)
            FGFR3_active_xy = xy_stack(fgfr3["clusters"]["com_x"][:reduce_size], fgfr3["clusters"]["com_y"][:reduce_size], pixel_size=pixel_size)
            FGFR4_active_xy = xy_stack(fgfr4["clusters"]["com_x"][:reduce_size], fgfr4["clusters"]["com_y"][:reduce_size], pixel_size=pixel_size)
            # store the neighbor xy-localizations of one measurement in lists
            neighbors_active_FGFR1, neighbors_active_FGFR2, neighbors_active_FGFR3, neighbors_active_FGFR4 = [], [], [], []
            neighbors_active_FGFR1.append(FGFR1_active_xy[:reduce_size])
            neighbors_active_FGFR2.append(FGFR2_active_xy[:reduce_size])
            neighbors_active_FGFR3.append(FGFR3_active_xy[:reduce_size])
            neighbors_active_FGFR4.append(FGFR4_active_xy[:reduce_size])
            FGFR1_distances = []  # stores distances FGFR1->1/2/3/4 of one measurement
            grid_11 = GridNNSearch(FGFR1_active_xy[:reduce_size], neighbors_active_FGFR1,
                                   int(np.floor(math.sqrt(len(neighbors_active_FGFR1[0])))), n_pixels, pixel_size)
            nn_11, _ = grid_11.get_nn_distances()
            grid_12 = GridNNSearch(FGFR1_active_xy[:reduce_size], neighbors_active_FGFR2,
                                   int(np.floor(math.sqrt(len(neighbors_active_FGFR2[0])))), n_pixels, pixel_size)
            nn_12, _ = grid_12.get_nn_distances()
            grid_13 = GridNNSearch(FGFR1_active_xy[:reduce_size], neighbors_active_FGFR3,
                                   int(np.floor(math.sqrt(len(neighbors_active_FGFR3[0])))), n_pixels, pixel_size)
            nn_13, _ = grid_13.get_nn_distances()
            grid_14 = GridNNSearch(FGFR1_active_xy[:reduce_size], neighbors_active_FGFR4,
                                   int(np.floor(math.sqrt(len(neighbors_active_FGFR4[0])))), n_pixels, pixel_size)
            nn_14, _ = grid_14.get_nn_distances()
            FGFR1_distances.append(nn_11); FGFR1_distances.append(nn_12)
            FGFR1_distances.append(nn_13); FGFR1_distances.append(nn_14)
            FGFR1_nn_distances_active.append(FGFR1_distances)
            pbar_B.update(1)
            FGFR2_distances = []
            grid_21 = GridNNSearch(FGFR2_active_xy[:reduce_size], neighbors_active_FGFR1,
                                   int(np.floor(math.sqrt(len(neighbors_active_FGFR1[0])))), n_pixels, pixel_size)
            nn_21, _ = grid_21.get_nn_distances()
            grid_22 = GridNNSearch(FGFR2_active_xy[:reduce_size], neighbors_active_FGFR2,
                                   int(np.floor(math.sqrt(len(neighbors_active_FGFR2[0])))), n_pixels, pixel_size)
            nn_22, _ = grid_22.get_nn_distances()
            grid_23 = GridNNSearch(FGFR2_active_xy[:reduce_size], neighbors_active_FGFR3,
                                   int(np.floor(math.sqrt(len(neighbors_active_FGFR3[0])))), n_pixels, pixel_size)
            nn_23, _ = grid_23.get_nn_distances()
            grid_24 = GridNNSearch(FGFR2_active_xy[:reduce_size], neighbors_active_FGFR4,
                                   int(np.floor(math.sqrt(len(neighbors_active_FGFR4[0])))), n_pixels, pixel_size)
            nn_24, _ = grid_24.get_nn_distances()
            FGFR2_distances.append(nn_21); FGFR2_distances.append(nn_22)
            FGFR2_distances.append(nn_23); FGFR2_distances.append(nn_24)
            FGFR2_nn_distances_active.append(FGFR2_distances)    
            pbar_B.update(1)
            FGFR3_distances = []
            grid_31 = GridNNSearch(FGFR3_active_xy[:reduce_size], neighbors_active_FGFR1,
                                   int(np.floor(math.sqrt(len(neighbors_active_FGFR1[0])))), n_pixels, pixel_size)
            nn_31, _ = grid_31.get_nn_distances()
            grid_32 = GridNNSearch(FGFR3_active_xy[:reduce_size], neighbors_active_FGFR2,
                                   int(np.floor(math.sqrt(len(neighbors_active_FGFR2[0])))), n_pixels, pixel_size)
            nn_32, _ = grid_32.get_nn_distances()
            grid_33 = GridNNSearch(FGFR3_active_xy[:reduce_size], neighbors_active_FGFR3,
                                   int(np.floor(math.sqrt(len(neighbors_active_FGFR3[0])))), n_pixels, pixel_size)
            nn_33, _ = grid_33.get_nn_distances()
            grid_34 = GridNNSearch(FGFR3_active_xy[:reduce_size], neighbors_active_FGFR4,
                                   int(np.floor(math.sqrt(len(neighbors_active_FGFR4[0])))), n_pixels, pixel_size)
            nn_34, _ = grid_34.get_nn_distances()
            FGFR3_distances.append(nn_31); FGFR3_distances.append(nn_32)
            FGFR3_distances.append(nn_33); FGFR3_distances.append(nn_34)
            FGFR3_nn_distances_active.append(FGFR3_distances)
            pbar_B.update(1)
            FGFR4_distances = []
            grid_41 = GridNNSearch(FGFR4_active_xy[:reduce_size], neighbors_active_FGFR1,
                                   int(np.floor(math.sqrt(len(neighbors_active_FGFR1[0])))), n_pixels, pixel_size)
            nn_41, _ = grid_41.get_nn_distances()
            grid_42 = GridNNSearch(FGFR4_active_xy[:reduce_size], neighbors_active_FGFR2,
                                   int(np.floor(math.sqrt(len(neighbors_active_FGFR2[0])))), n_pixels, pixel_size)
            nn_42, _ = grid_42.get_nn_distances()
            grid_43 = GridNNSearch(FGFR4_active_xy[:reduce_size], neighbors_active_FGFR3,
                                   int(np.floor(math.sqrt(len(neighbors_active_FGFR3[0])))), n_pixels, pixel_size)
            nn_43, _ = grid_43.get_nn_distances()
            grid_44 = GridNNSearch(FGFR4_active_xy[:reduce_size], neighbors_active_FGFR4,
                                   int(np.floor(math.sqrt(len(neighbors_active_FGFR4[0])))), n_pixels, pixel_size)
            nn_44 = grid_44.get_nn_distances()[0]
            FGFR4_distances.append(nn_41); FGFR4_distances.append(nn_42)
            FGFR4_distances.append(nn_43); FGFR4_distances.append(nn_44)
            FGFR4_nn_distances_active.append(FGFR4_distances)
            pbar_B.update(1)
            pbar.update(1)
            
print("Total execution took {0:.2f} s.".format(time.time()-start))

#### Plot the results

- Scatter plots with error bars display the mean values & the mean errors of one measurement condition, a direct comparison between not active and active is possible here.
- Choose a bin size in nm!
- Set single_neighbor_one to True to display single measurements as histograms.
- The x-axis point at e.g. 50 corresponds to a range between 0-50 nm.
- Be careful about the xaxis-ranges while comparing! Change the displayed x-axis range via the variable "x_axes_display_range" of the plotting function, consider this every time the plotting function is called that displayes the figure with four subplots:
    - x_axes_display_range = max_distance -> sets the range to the maximum distance recorded based on all measurements
    - x_axes_display_range = None -> displays the maximum values per subplot (the x-ranges differ in one figure) 
    - x_axes_display_range = integer -> all subplots are displayed to this integer (values that are higher are not displayed, the x-ranges are the same in one figure)
    - x_axes_display_range = max_distance_mi -> sets the range to the maximum distance recorded in one measurement (only available for the single measurement plots, the x-ranges are the same in one figure)
    - Reasonable settings for the x_axes_display_range are set by default.
- The y-axis range can be set for all plots as a float (e.g. 0.4) or to "max" (this finds the maximum mean y-val and sets all axis to it+offset for the error bars).

In [ ]:
bin_size_neighbor_one = 50  # set a bin size, nm
single_neighbor_one = True  # True/False
y_max_value_one = "max"  # "max" -> finds maximum value automatically, or set it to a value, e.g. 0.4

In [ ]:
def plot_hist(x_a, x_b, x_c, x_d, single=False, max_distance=None, active=False, subplot_titles=(),
              bin_size=50, measurement_id="", y_axes_display_range=None, x_axes_display_range=None):
    fig = make_subplots(rows=2, cols=2, subplot_titles=subplot_titles)
    trace0 = go.Histogram(x=x_a, histnorm="probability", xbins=dict(start=0, end=max_distance, size=bin_size), marker=dict(color=target_colors[0]), name=subplot_titles[0])  
    trace1 = go.Histogram(x=x_b, histnorm="probability", xbins=dict(start=0, end=max_distance, size=bin_size), marker=dict(color=target_colors[1]), name=subplot_titles[1])
    trace2 = go.Histogram(x=x_c, histnorm="probability", xbins=dict(start=0, end=max_distance, size=bin_size), marker=dict(color=target_colors[2]), name=subplot_titles[2])
    trace3 = go.Histogram(x=x_d, histnorm="probability", xbins=dict(start=0, end=max_distance, size=bin_size), marker=dict(color=target_colors[3]), name=subplot_titles[3])
    fig.append_trace(trace0, row=1, col=1)
    fig.append_trace(trace1, row=1, col=2)
    fig.append_trace(trace2, row=2, col=1)
    fig.append_trace(trace3, row=2, col=2)
    fig.update_xaxes(range=[0, x_axes_display_range])
    fig.update_yaxes(range=[0, y_axes_display_range])
    measurement_space = "single measurement " + measurement_id if single else "all measurements"
    measurement_condition = condition_names[1] if active else condition_names[0]
    fig.update_layout(title_text= measurement_condition + " " + measurement_space, template=background_template)
    fig.show() 

def plot_hist_mean(x_a_nactive, x_b_nactive, x_c_nactive, x_d_nactive,
                   x_a_error_nactive, x_b_error_nactive, x_c_error_nactive, x_d_error_nactive,
                   x_a_active, x_b_active, x_c_active, x_d_active,
                   x_a_error_active, x_b_error_active, x_c_error_active, x_d_error_active,
                   bins, subplot_titles, y_axes_display_range=None, x_axes_display_range=None):
    fig = make_subplots(rows=2, cols=2, subplot_titles=subplot_titles)
    trace0na = go.Scatter(x=bins, y=x_a_nactive, error_y=dict(type="data", array=x_a_error_nactive, visible=True),
                        line=dict(color=condition_colors[0]), name=subplot_titles[0] +" "+condition_names[0])
    trace0a = go.Scatter(x=bins, y=x_a_active, error_y=dict(type="data", array=x_a_error_active, visible=True),
                        line=dict(color=condition_colors[1]), name=subplot_titles[0] +" "+condition_names[1])
    trace1na = go.Scatter(x=bins, y=x_b_nactive, error_y=dict(type="data", array=x_b_error_nactive, visible=True),
                        line=dict(color=condition_colors[0]), name=subplot_titles[1] +" "+condition_names[0])
    trace1a = go.Scatter(x=bins, y=x_b_active, error_y=dict(type="data", array=x_b_error_active, visible=True),
                        line=dict(color=condition_colors[1]), name=subplot_titles[1] +" "+condition_names[1])
    trace2na = go.Scatter(x=bins, y=x_c_nactive, error_y=dict(type="data", array=x_c_error_nactive, visible=True),
                        line=dict(color=condition_colors[0]), name=subplot_titles[2] +" "+condition_names[0])
    trace2a = go.Scatter(x=bins, y=x_c_active, error_y=dict(type="data", array=x_c_error_active, visible=True),
                        line=dict(color=condition_colors[1]), name=subplot_titles[2] +" "+condition_names[1])
    trace3na = go.Scatter(x=bins, y=x_d_nactive, error_y=dict(type="data", array=x_d_error_nactive, visible=True),
                        line=dict(color=condition_colors[0]), name=subplot_titles[3] +" "+condition_names[0])
    trace3a = go.Scatter(x=bins, y=x_d_active, error_y=dict(type="data", array=x_d_error_active, visible=True),
                        line=dict(color=condition_colors[1]), name=subplot_titles[3] + " "+condition_names[1])
    fig.append_trace(trace0na, row=1, col=1)
    fig.append_trace(trace0a, row=1, col=1)
    fig.append_trace(trace1na, row=1, col=2)
    fig.append_trace(trace1a, row=1, col=2)
    fig.append_trace(trace2na, row=2, col=1)
    fig.append_trace(trace2a, row=2, col=1)
    fig.append_trace(trace3na, row=2, col=2)
    fig.append_trace(trace3a, row=2, col=2)
    fig.update_xaxes(range=[0, x_axes_display_range])
    fig.update_yaxes(range=[0, y_axes_display_range])
    fig.update_layout(template=background_template)
    fig.show()
    
def calc_hist_values(measurement, bins):
    hist_val = np.histogram(measurement, bins=bins)
    hist_sum = np.sum(hist_val[0])
    hist_val = np.divide(hist_val[0], hist_sum)
    return hist_val

def calc_mean_hist_values(FGFRXY_distances, bins):
    if len(FGFRXY_distances) > 1:
        hist_vals = []
        for measurement in FGFRXY_distances:
            hist_val = calc_hist_values(measurement, bins)
            hist_vals.append(hist_val)
        FGFRXY_hist_mean = np.mean(hist_vals, axis=0)
        FGFRXY_hist_mean_error = np.std(hist_vals, axis=0, ddof=1)/math.sqrt(len(FGFRXY_distances))
    else:
        hist_val = calc_hist_values(FGFRXY_distances[0], bins)
        FGFRXY_hist_mean = hist_val
        FGFRXY_hist_mean_error = [0]*len(hist_val)
    return FGFRXY_hist_mean, FGFRXY_hist_mean_error


FGFR11_distances_nactive = [measurement[0] for measurement in FGFR1_nn_distances_nactive]
FGFR12_distances_nactive = [measurement[1] for measurement in FGFR1_nn_distances_nactive]
FGFR13_distances_nactive = [measurement[2] for measurement in FGFR1_nn_distances_nactive]
FGFR14_distances_nactive = [measurement[3] for measurement in FGFR1_nn_distances_nactive]
FGFR21_distances_nactive = [measurement[0] for measurement in FGFR2_nn_distances_nactive]
FGFR22_distances_nactive = [measurement[1] for measurement in FGFR2_nn_distances_nactive]
FGFR23_distances_nactive = [measurement[2] for measurement in FGFR2_nn_distances_nactive]
FGFR24_distances_nactive = [measurement[3] for measurement in FGFR2_nn_distances_nactive]
FGFR31_distances_nactive = [measurement[0] for measurement in FGFR3_nn_distances_nactive]
FGFR32_distances_nactive = [measurement[1] for measurement in FGFR3_nn_distances_nactive]
FGFR33_distances_nactive = [measurement[2] for measurement in FGFR3_nn_distances_nactive]
FGFR34_distances_nactive = [measurement[3] for measurement in FGFR3_nn_distances_nactive]
FGFR41_distances_nactive = [measurement[0] for measurement in FGFR4_nn_distances_nactive]
FGFR42_distances_nactive = [measurement[1] for measurement in FGFR4_nn_distances_nactive]
FGFR43_distances_nactive = [measurement[2] for measurement in FGFR4_nn_distances_nactive]
FGFR44_distances_nactive = [measurement[3] for measurement in FGFR4_nn_distances_nactive]

FGFR11_distances_active = [measurement[0] for measurement in FGFR1_nn_distances_active]
FGFR12_distances_active = [measurement[1] for measurement in FGFR1_nn_distances_active]
FGFR13_distances_active = [measurement[2] for measurement in FGFR1_nn_distances_active]
FGFR14_distances_active = [measurement[3] for measurement in FGFR1_nn_distances_active]
FGFR21_distances_active = [measurement[0] for measurement in FGFR2_nn_distances_active]
FGFR22_distances_active = [measurement[1] for measurement in FGFR2_nn_distances_active]
FGFR23_distances_active = [measurement[2] for measurement in FGFR2_nn_distances_active]
FGFR24_distances_active = [measurement[3] for measurement in FGFR2_nn_distances_active]
FGFR31_distances_active = [measurement[0] for measurement in FGFR3_nn_distances_active]
FGFR32_distances_active = [measurement[1] for measurement in FGFR3_nn_distances_active]
FGFR33_distances_active = [measurement[2] for measurement in FGFR3_nn_distances_active]
FGFR34_distances_active = [measurement[3] for measurement in FGFR3_nn_distances_active]
FGFR41_distances_active = [measurement[0] for measurement in FGFR4_nn_distances_active]
FGFR42_distances_active = [measurement[1] for measurement in FGFR4_nn_distances_active]
FGFR43_distances_active = [measurement[2] for measurement in FGFR4_nn_distances_active]
FGFR44_distances_active = [measurement[3] for measurement in FGFR4_nn_distances_active]

all_FGFR11_distances_nactive = [j for i in FGFR11_distances_nactive for j in i]
all_FGFR12_distances_nactive = [j for i in FGFR12_distances_nactive for j in i]
all_FGFR13_distances_nactive = [j for i in FGFR13_distances_nactive for j in i]
all_FGFR14_distances_nactive = [j for i in FGFR14_distances_nactive for j in i]
all_FGFR21_distances_nactive = [j for i in FGFR21_distances_nactive for j in i]
all_FGFR22_distances_nactive = [j for i in FGFR22_distances_nactive for j in i]
all_FGFR23_distances_nactive = [j for i in FGFR23_distances_nactive for j in i]
all_FGFR24_distances_nactive = [j for i in FGFR24_distances_nactive for j in i]
all_FGFR31_distances_nactive = [j for i in FGFR31_distances_nactive for j in i]
all_FGFR32_distances_nactive = [j for i in FGFR32_distances_nactive for j in i]
all_FGFR33_distances_nactive = [j for i in FGFR33_distances_nactive for j in i]
all_FGFR34_distances_nactive = [j for i in FGFR34_distances_nactive for j in i]
all_FGFR41_distances_nactive = [j for i in FGFR41_distances_nactive for j in i]
all_FGFR42_distances_nactive = [j for i in FGFR42_distances_nactive for j in i]
all_FGFR43_distances_nactive = [j for i in FGFR43_distances_nactive for j in i]
all_FGFR44_distances_nactive = [j for i in FGFR44_distances_nactive for j in i]

all_FGFR11_distances_active = [j for i in FGFR11_distances_active for j in i]
all_FGFR12_distances_active = [j for i in FGFR12_distances_active for j in i]
all_FGFR13_distances_active = [j for i in FGFR13_distances_active for j in i]
all_FGFR14_distances_active = [j for i in FGFR14_distances_active for j in i]
all_FGFR21_distances_active = [j for i in FGFR21_distances_active for j in i]
all_FGFR22_distances_active = [j for i in FGFR22_distances_active for j in i]
all_FGFR23_distances_active = [j for i in FGFR23_distances_active for j in i]
all_FGFR24_distances_active = [j for i in FGFR24_distances_active for j in i]
all_FGFR31_distances_active = [j for i in FGFR31_distances_active for j in i]
all_FGFR32_distances_active = [j for i in FGFR32_distances_active for j in i]
all_FGFR33_distances_active = [j for i in FGFR33_distances_active for j in i]
all_FGFR34_distances_active = [j for i in FGFR34_distances_active for j in i]
all_FGFR41_distances_active = [j for i in FGFR41_distances_active for j in i]
all_FGFR42_distances_active = [j for i in FGFR42_distances_active for j in i]
all_FGFR43_distances_active = [j for i in FGFR43_distances_active for j in i]
all_FGFR44_distances_active = [j for i in FGFR44_distances_active for j in i]

max_distance = max(all_FGFR11_distances_nactive+all_FGFR12_distances_nactive+all_FGFR13_distances_nactive+all_FGFR14_distances_nactive
                   +all_FGFR21_distances_nactive+all_FGFR22_distances_nactive+all_FGFR23_distances_nactive+all_FGFR24_distances_nactive
                   +all_FGFR31_distances_nactive+all_FGFR32_distances_nactive+all_FGFR33_distances_nactive+all_FGFR34_distances_nactive
                   +all_FGFR41_distances_nactive+all_FGFR42_distances_nactive+all_FGFR43_distances_nactive+all_FGFR44_distances_nactive
                   +all_FGFR11_distances_active+all_FGFR12_distances_active+all_FGFR13_distances_active+all_FGFR14_distances_active
                   +all_FGFR21_distances_active+all_FGFR22_distances_active+all_FGFR23_distances_active+all_FGFR24_distances_active
                   +all_FGFR31_distances_active+all_FGFR32_distances_active+all_FGFR33_distances_active+all_FGFR34_distances_active
                   +all_FGFR41_distances_active+all_FGFR42_distances_active+all_FGFR43_distances_active+all_FGFR44_distances_active)

max_bin_neighbor_one = int(np.ceil(max_distance/bin_size_neighbor_one))
bins_neighbor_one = [bin_size_neighbor_one*i for i in range(max_bin_neighbor_one+1)]
bins_plot_neighbor_one = [bin_size_neighbor_one*i for i in range(1,max_bin_neighbor_one+2)]
    
# all measurements not active 1->x
FGFR11_nactive_hist_mean, FGFR11_nactive_hist_mean_error = calc_mean_hist_values(FGFR11_distances_nactive, bins_neighbor_one)
FGFR12_nactive_hist_mean, FGFR12_nactive_hist_mean_error = calc_mean_hist_values(FGFR12_distances_nactive, bins_neighbor_one)
FGFR13_nactive_hist_mean, FGFR13_nactive_hist_mean_error = calc_mean_hist_values(FGFR13_distances_nactive, bins_neighbor_one)
FGFR14_nactive_hist_mean, FGFR14_nactive_hist_mean_error = calc_mean_hist_values(FGFR14_distances_nactive, bins_neighbor_one)
# all measurements active 1->x
FGFR11_active_hist_mean, FGFR11_active_hist_mean_error = calc_mean_hist_values(FGFR11_distances_active, bins_neighbor_one)
FGFR12_active_hist_mean, FGFR12_active_hist_mean_error = calc_mean_hist_values(FGFR12_distances_active, bins_neighbor_one)
FGFR13_active_hist_mean, FGFR13_active_hist_mean_error = calc_mean_hist_values(FGFR13_distances_active, bins_neighbor_one)
FGFR14_active_hist_mean, FGFR14_active_hist_mean_error = calc_mean_hist_values(FGFR14_distances_active, bins_neighbor_one)
# all measurements not active 2->x
FGFR21_nactive_hist_mean, FGFR21_nactive_hist_mean_error = calc_mean_hist_values(FGFR21_distances_nactive, bins_neighbor_one)
FGFR22_nactive_hist_mean, FGFR22_nactive_hist_mean_error = calc_mean_hist_values(FGFR22_distances_nactive, bins_neighbor_one)
FGFR23_nactive_hist_mean, FGFR23_nactive_hist_mean_error = calc_mean_hist_values(FGFR23_distances_nactive, bins_neighbor_one)
FGFR24_nactive_hist_mean, FGFR24_nactive_hist_mean_error = calc_mean_hist_values(FGFR24_distances_nactive, bins_neighbor_one)
# all measurements active 2->x
FGFR21_active_hist_mean, FGFR21_active_hist_mean_error = calc_mean_hist_values(FGFR21_distances_active, bins_neighbor_one)
FGFR22_active_hist_mean, FGFR22_active_hist_mean_error = calc_mean_hist_values(FGFR22_distances_active, bins_neighbor_one)
FGFR23_active_hist_mean, FGFR23_active_hist_mean_error = calc_mean_hist_values(FGFR23_distances_active, bins_neighbor_one)
FGFR24_active_hist_mean, FGFR24_active_hist_mean_error = calc_mean_hist_values(FGFR24_distances_active, bins_neighbor_one)
# all measurements not active 3->x
FGFR31_nactive_hist_mean, FGFR31_nactive_hist_mean_error = calc_mean_hist_values(FGFR31_distances_nactive, bins_neighbor_one)
FGFR32_nactive_hist_mean, FGFR32_nactive_hist_mean_error = calc_mean_hist_values(FGFR32_distances_nactive, bins_neighbor_one)
FGFR33_nactive_hist_mean, FGFR33_nactive_hist_mean_error = calc_mean_hist_values(FGFR33_distances_nactive, bins_neighbor_one)
FGFR34_nactive_hist_mean, FGFR34_nactive_hist_mean_error = calc_mean_hist_values(FGFR34_distances_nactive, bins_neighbor_one)
# all measurements active 3->x
FGFR31_active_hist_mean, FGFR31_active_hist_mean_error = calc_mean_hist_values(FGFR31_distances_active, bins_neighbor_one)
FGFR32_active_hist_mean, FGFR32_active_hist_mean_error = calc_mean_hist_values(FGFR32_distances_active, bins_neighbor_one)
FGFR33_active_hist_mean, FGFR33_active_hist_mean_error = calc_mean_hist_values(FGFR33_distances_active, bins_neighbor_one)
FGFR34_active_hist_mean, FGFR34_active_hist_mean_error = calc_mean_hist_values(FGFR34_distances_active, bins_neighbor_one)
# all measurements not active 4->x
FGFR41_nactive_hist_mean, FGFR41_nactive_hist_mean_error = calc_mean_hist_values(FGFR41_distances_nactive, bins_neighbor_one)
FGFR42_nactive_hist_mean, FGFR42_nactive_hist_mean_error = calc_mean_hist_values(FGFR42_distances_nactive, bins_neighbor_one)
FGFR43_nactive_hist_mean, FGFR43_nactive_hist_mean_error = calc_mean_hist_values(FGFR43_distances_nactive, bins_neighbor_one)
FGFR44_nactive_hist_mean, FGFR44_nactive_hist_mean_error = calc_mean_hist_values(FGFR44_distances_nactive, bins_neighbor_one)
# all measurements active 4->x
FGFR41_active_hist_mean, FGFR41_active_hist_mean_error = calc_mean_hist_values(FGFR41_distances_active, bins_neighbor_one)
FGFR42_active_hist_mean, FGFR42_active_hist_mean_error = calc_mean_hist_values(FGFR42_distances_active, bins_neighbor_one)
FGFR43_active_hist_mean, FGFR43_active_hist_mean_error = calc_mean_hist_values(FGFR43_distances_active, bins_neighbor_one)
FGFR44_active_hist_mean, FGFR44_active_hist_mean_error = calc_mean_hist_values(FGFR44_distances_active, bins_neighbor_one)

# find y-axis range
if y_max_value_one == "max":
    y_max_nn_one = max(np.append(FGFR11_nactive_hist_mean, [FGFR12_nactive_hist_mean, FGFR13_nactive_hist_mean, FGFR14_nactive_hist_mean,
                                        FGFR21_nactive_hist_mean, FGFR22_nactive_hist_mean, FGFR23_nactive_hist_mean,
                                        FGFR24_nactive_hist_mean, FGFR31_nactive_hist_mean, FGFR32_nactive_hist_mean,
                                        FGFR33_nactive_hist_mean, FGFR34_nactive_hist_mean, FGFR41_nactive_hist_mean,
                                        FGFR42_nactive_hist_mean, FGFR43_nactive_hist_mean, FGFR44_nactive_hist_mean,
                                        FGFR11_active_hist_mean, FGFR12_active_hist_mean, FGFR13_active_hist_mean,
                                        FGFR14_active_hist_mean, FGFR21_active_hist_mean, FGFR22_active_hist_mean,
                                        FGFR23_active_hist_mean, FGFR24_active_hist_mean, FGFR31_active_hist_mean,
                                        FGFR32_active_hist_mean, FGFR33_active_hist_mean, FGFR34_active_hist_mean,
                                        FGFR41_active_hist_mean, FGFR42_active_hist_mean, FGFR43_active_hist_mean,
                                        FGFR44_active_hist_mean], axis=None))
    y_max_nn_one += 0.1
else: 
    y_max_nn_one = y_max_value_one
# get plot descriptions
plot_descs = []
for name_A in target_names:
    plot_desc = [name_A + "->" + name_B for name_B in target_names]
    plot_descs.append(plot_desc)

plot_hist_mean(FGFR11_nactive_hist_mean, FGFR12_nactive_hist_mean, FGFR13_nactive_hist_mean, FGFR14_nactive_hist_mean,
              FGFR11_nactive_hist_mean_error, FGFR12_nactive_hist_mean_error, FGFR13_nactive_hist_mean_error, FGFR14_nactive_hist_mean_error,
              FGFR11_active_hist_mean, FGFR12_active_hist_mean, FGFR13_active_hist_mean, FGFR14_active_hist_mean,
              FGFR11_active_hist_mean_error, FGFR12_active_hist_mean_error, FGFR13_active_hist_mean_error, FGFR14_active_hist_mean_error,
              bins_plot_neighbor_one, plot_descs[0], y_axes_display_range=y_max_nn_one, x_axes_display_range=800)  # max_distance, None, integer
    
plot_hist_mean(FGFR21_nactive_hist_mean, FGFR22_nactive_hist_mean, FGFR23_nactive_hist_mean, FGFR24_nactive_hist_mean,
              FGFR21_nactive_hist_mean_error, FGFR22_nactive_hist_mean_error, FGFR23_nactive_hist_mean_error, FGFR24_nactive_hist_mean_error,
              FGFR21_active_hist_mean, FGFR22_active_hist_mean, FGFR23_active_hist_mean, FGFR24_active_hist_mean,
              FGFR21_active_hist_mean_error, FGFR22_active_hist_mean_error, FGFR23_active_hist_mean_error, FGFR24_active_hist_mean_error,
              bins_plot_neighbor_one, plot_descs[1], y_axes_display_range=y_max_nn_one, x_axes_display_range=800) # max_distance, None, integer

plot_hist_mean(FGFR31_nactive_hist_mean, FGFR32_nactive_hist_mean, FGFR33_nactive_hist_mean, FGFR34_nactive_hist_mean,
              FGFR31_nactive_hist_mean_error, FGFR32_nactive_hist_mean_error, FGFR33_nactive_hist_mean_error, FGFR34_nactive_hist_mean_error,
              FGFR31_active_hist_mean, FGFR32_active_hist_mean, FGFR33_active_hist_mean, FGFR34_active_hist_mean,
              FGFR31_active_hist_mean_error, FGFR32_active_hist_mean_error, FGFR33_active_hist_mean_error, FGFR34_active_hist_mean_error,
              bins_plot_neighbor_one, plot_descs[2], y_axes_display_range=y_max_nn_one, x_axes_display_range=800) # max_distance, None, integer

plot_hist_mean(FGFR41_nactive_hist_mean, FGFR42_nactive_hist_mean, FGFR43_nactive_hist_mean, FGFR44_nactive_hist_mean,
              FGFR41_nactive_hist_mean_error, FGFR42_nactive_hist_mean_error, FGFR43_nactive_hist_mean_error, FGFR44_nactive_hist_mean_error,
              FGFR41_active_hist_mean, FGFR42_active_hist_mean, FGFR43_active_hist_mean, FGFR44_active_hist_mean,
              FGFR41_active_hist_mean_error, FGFR42_active_hist_mean_error, FGFR43_active_hist_mean_error, FGFR44_active_hist_mean_error,
              bins_plot_neighbor_one, plot_descs[3], y_axes_display_range=y_max_nn_one, x_axes_display_range=800) # max_distance, None, integer

if single_neighbor_one:
    # single measurements not active 1->x
    for c, measurement in enumerate(FGFR1_nn_distances_nactive, 1):
        max_distance_mi = max(measurement[0]+measurement[1]+measurement[2]+measurement[3])
        plot_hist(measurement[0], measurement[1], measurement[2],
          measurement[3], single=True, max_distance=max_distance, active=False,
          subplot_titles=plot_descs[0], bin_size=bin_size_neighbor_one,
          measurement_id = str(c), y_axes_display_range=y_max_nn_one, x_axes_display_range=max_distance_mi)  # max_distance, None, integer, max_distance_mi
        
    # single measurements active 1->x
    for c, measurement in enumerate(FGFR1_nn_distances_active, 1):
        max_distance_mi = max(measurement[0]+measurement[1]+measurement[2]+measurement[3])
        plot_hist(measurement[0], measurement[1], measurement[2],
          measurement[3], single=True, max_distance=max_distance, active=True,
          subplot_titles=plot_descs[0], bin_size=bin_size_neighbor_one,
          measurement_id = str(c), y_axes_display_range=y_max_nn_one, x_axes_display_range=max_distance_mi)  # max_distance, None, integer, max_distance_mi
        
    # single measurements not active 2->x
    for c, measurement in enumerate(FGFR2_nn_distances_nactive, 1):
        max_distance_mi = max(measurement[0]+measurement[1]+measurement[2]+measurement[3])
        plot_hist(measurement[0], measurement[1], measurement[2],
          measurement[3], single=True, max_distance=max_distance, active=False,
          subplot_titles=plot_descs[1], bin_size=bin_size_neighbor_one,
          measurement_id = str(c), y_axes_display_range=y_max_nn_one, x_axes_display_range=max_distance_mi)  # max_distance, None, integer, max_distance_mi
    # single measurements active 2->x
    for c, measurement in enumerate(FGFR2_nn_distances_active, 1):
        max_distance_mi = max(measurement[0]+measurement[1]+measurement[2]+measurement[3])
        plot_hist(measurement[0], measurement[1], measurement[2],
          measurement[3], single=True, max_distance=max_distance, active=True,
          subplot_titles=plot_descs[1], bin_size=bin_size_neighbor_one,
          measurement_id = str(c), y_axes_display_range=y_max_nn_one, x_axes_display_range=max_distance_mi)  # max_distance, None, integer, max_distance_mi
        
    # single measurements not active 3->x
    for c, measurement in enumerate(FGFR3_nn_distances_nactive,1 ):
        max_distance_mi = max(measurement[0]+measurement[1]+measurement[2]+measurement[3])
        plot_hist(measurement[0], measurement[1], measurement[2],
          measurement[3], single=True, max_distance=max_distance, active=False,
          subplot_titles=plot_descs[2], bin_size=bin_size_neighbor_one,
          measurement_id = str(c), y_axes_display_range=y_max_nn_one, x_axes_display_range=max_distance_mi)  # max_distance, None, integer, max_distance_mi
    # single measurements active 3->x
    for c, measurement in enumerate(FGFR3_nn_distances_active, 1):
        max_distance_mi = max(measurement[0]+measurement[1]+measurement[2]+measurement[3])
        plot_hist(measurement[0], measurement[1], measurement[2],
          measurement[3], single=True, max_distance=max_distance, active=True,
          subplot_titles=plot_descs[2], bin_size=bin_size_neighbor_one,
          measurement_id = str(c), y_axes_display_range=y_max_nn_one, x_axes_display_range=max_distance_mi)  # max_distance, None, integer, max_distance_mi
        
    # single measurements not active 4->x
    for c, measurement in enumerate(FGFR4_nn_distances_nactive, 1):
        max_distance_mi = max(measurement[0]+measurement[1]+measurement[2]+measurement[3])
        plot_hist(measurement[0], measurement[1], measurement[2],
          measurement[3], single=True, max_distance=max_distance, active=False,
          subplot_titles=plot_descs[3], bin_size=bin_size_neighbor_one,
          measurement_id = str(c), y_axes_display_range=y_max_nn_one, x_axes_display_range=max_distance_mi)  # max_distance, None, integer, max_distance_mi
    # single measurements active 4->x
    for c, measurement in enumerate(FGFR4_nn_distances_active, 1):
        max_distance_mi = max(measurement[0]+measurement[1]+measurement[2]+measurement[3])
        plot_hist(measurement[0], measurement[1], measurement[2],
          measurement[3], single=True, max_distance=max_distance, active=True,
          subplot_titles=plot_descs[3], bin_size=bin_size_neighbor_one,
          measurement_id = str(c), y_axes_display_range=y_max_nn_one, x_axes_display_range=max_distance_mi)  # max_distance, None, integer, max_distance_mi

### Amount of nearest neighbor distances less or equal to a threshold

- Set a threshold distance in nm, all distances <= threshold are accepted. The fraction of accepted distances is calculated and displayed as a barplots.

#### Execute the analysis

In [ ]:
threshold_neighbor_one_below = 50  # accepted distance <= threshold

In [ ]:
def calc_distance_below(distances, threshold):
    """
    :return: The % of distances below a threshold per measurement, list of distances below a threshold per measurement.
    """
    count_distances = len(distances)
    count_distances_below = len([i for i in distances if i <= threshold])
    distances_below = [i for i in distances if i <= threshold]
    distances_below_norm = count_distances_below / count_distances
    return distances_below_norm, distances_below

def calc_mean_error(FGFR_below):
    if len(FGFR_below) > 1:
        FGFR_below_mean = np.mean(FGFR_below, axis=0)
        FGFR_below_mean_error = np.std(FGFR_below, axis=0, ddof=1)/math.sqrt(len(FGFR_below))
    else:
        FGFR_below_mean = FGFR_below[0]
        FGFR_below_mean_error = [0]*len(FGFR_below[0])
    return FGFR_below_mean, FGFR_below_mean_error

def get_distance_below_fraction(XA, XB, XC, XD, threshold):
    distances_below_percent = []
    distances_below_XA = []
    distances_below_XB = []
    distances_below_XC = []
    distances_below_XD = []
    for xa, xb, xc, xd in zip(XA, XB, XC, XD):
        distances_below_percent_mi = []
        distances_below_percent_mi.append(calc_distance_below(xa, threshold)[0])
        distances_below_percent_mi.append(calc_distance_below(xb, threshold)[0])
        distances_below_percent_mi.append(calc_distance_below(xc, threshold)[0])
        distances_below_percent_mi.append(calc_distance_below(xd, threshold)[0])
        distances_below_percent.append(distances_below_percent_mi)
        distances_below_XA.append(calc_distance_below(xa, threshold)[1])
        distances_below_XB.append(calc_distance_below(xb, threshold)[1])
        distances_below_XC.append(calc_distance_below(xc, threshold)[1])
        distances_below_XD.append(calc_distance_below(xd, threshold)[1])  
    FGFR_below_mean, FGFR_below_mean_error = calc_mean_error(distances_below_percent)
    distances_below = []
    distances_below.append(distances_below_XA)
    distances_below.append(distances_below_XB)
    distances_below.append(distances_below_XC)
    distances_below.append(distances_below_XD)
    return FGFR_below_mean, FGFR_below_mean_error, distances_below

# mean values, not active
distances_below_nactive_FGFR1X_mean, distances_below_nactive_FGFR1X_mean_error, distances_below_nactive_FGFR1X = get_distance_below_fraction(FGFR11_distances_nactive, FGFR12_distances_nactive, FGFR13_distances_nactive, FGFR14_distances_nactive, threshold_neighbor_one_below)
distances_below_nactive_FGFR2X_mean, distances_below_nactive_FGFR2X_mean_error, distances_below_nactive_FGFR2X = get_distance_below_fraction(FGFR21_distances_nactive, FGFR22_distances_nactive, FGFR23_distances_nactive, FGFR24_distances_nactive, threshold_neighbor_one_below)
distances_below_nactive_FGFR3X_mean, distances_below_nactive_FGFR3X_mean_error, distances_below_nactive_FGFR3X = get_distance_below_fraction(FGFR31_distances_nactive, FGFR32_distances_nactive, FGFR33_distances_nactive, FGFR34_distances_nactive, threshold_neighbor_one_below)
distances_below_nactive_FGFR4X_mean, distances_below_nactive_FGFR4X_mean_error, distances_below_nactive_FGFR4X = get_distance_below_fraction(FGFR41_distances_nactive, FGFR42_distances_nactive, FGFR43_distances_nactive, FGFR44_distances_nactive, threshold_neighbor_one_below)
# mean values, active
distances_below_active_FGFR1X_mean, distances_below_active_FGFR1X_mean_error, distances_below_active_FGFR1X = get_distance_below_fraction(FGFR11_distances_active, FGFR12_distances_active, FGFR13_distances_active, FGFR14_distances_active, threshold_neighbor_one_below)
distances_below_active_FGFR2X_mean, distances_below_active_FGFR2X_mean_error, distances_below_active_FGFR2X = get_distance_below_fraction(FGFR21_distances_active, FGFR22_distances_active, FGFR23_distances_active, FGFR24_distances_active, threshold_neighbor_one_below)
distances_below_active_FGFR3X_mean, distances_below_active_FGFR3X_mean_error, distances_below_active_FGFR3X = get_distance_below_fraction(FGFR31_distances_active, FGFR32_distances_active, FGFR33_distances_active, FGFR34_distances_active, threshold_neighbor_one_below)
distances_below_active_FGFR4X_mean, distances_below_active_FGFR4X_mean_error, distances_below_active_FGFR4X = get_distance_below_fraction(FGFR41_distances_active, FGFR42_distances_active, FGFR43_distances_active, FGFR44_distances_active, threshold_neighbor_one_below)
# single measurements, not active
FGFR11_distances_nactive_below = [calc_distance_below(measurement, threshold_neighbor_one_below) for measurement in FGFR11_distances_nactive]
FGFR12_distances_nactive_below = [calc_distance_below(measurement, threshold_neighbor_one_below) for measurement in FGFR12_distances_nactive]
FGFR13_distances_nactive_below = [calc_distance_below(measurement, threshold_neighbor_one_below) for measurement in FGFR13_distances_nactive]
FGFR14_distances_nactive_below = [calc_distance_below(measurement, threshold_neighbor_one_below) for measurement in FGFR14_distances_nactive]
FGFR21_distances_nactive_below = [calc_distance_below(measurement, threshold_neighbor_one_below) for measurement in FGFR21_distances_nactive]
FGFR22_distances_nactive_below = [calc_distance_below(measurement, threshold_neighbor_one_below) for measurement in FGFR22_distances_nactive]
FGFR23_distances_nactive_below = [calc_distance_below(measurement, threshold_neighbor_one_below) for measurement in FGFR23_distances_nactive]
FGFR24_distances_nactive_below = [calc_distance_below(measurement, threshold_neighbor_one_below) for measurement in FGFR24_distances_nactive]
FGFR31_distances_nactive_below = [calc_distance_below(measurement, threshold_neighbor_one_below) for measurement in FGFR31_distances_nactive]
FGFR32_distances_nactive_below = [calc_distance_below(measurement, threshold_neighbor_one_below) for measurement in FGFR32_distances_nactive]
FGFR33_distances_nactive_below = [calc_distance_below(measurement, threshold_neighbor_one_below) for measurement in FGFR33_distances_nactive]
FGFR34_distances_nactive_below = [calc_distance_below(measurement, threshold_neighbor_one_below) for measurement in FGFR34_distances_nactive]
FGFR41_distances_nactive_below = [calc_distance_below(measurement, threshold_neighbor_one_below) for measurement in FGFR41_distances_nactive]
FGFR42_distances_nactive_below = [calc_distance_below(measurement, threshold_neighbor_one_below) for measurement in FGFR42_distances_nactive]
FGFR43_distances_nactive_below = [calc_distance_below(measurement, threshold_neighbor_one_below) for measurement in FGFR43_distances_nactive]
FGFR44_distances_nactive_below = [calc_distance_below(measurement, threshold_neighbor_one_below) for measurement in FGFR44_distances_nactive]
# single measurements, active
FGFR11_distances_active_below = [calc_distance_below(measurement, threshold_neighbor_one_below) for measurement in FGFR11_distances_active]
FGFR12_distances_active_below = [calc_distance_below(measurement, threshold_neighbor_one_below) for measurement in FGFR12_distances_active]
FGFR13_distances_active_below = [calc_distance_below(measurement, threshold_neighbor_one_below) for measurement in FGFR13_distances_active]
FGFR14_distances_active_below = [calc_distance_below(measurement, threshold_neighbor_one_below) for measurement in FGFR14_distances_active]
FGFR21_distances_active_below = [calc_distance_below(measurement, threshold_neighbor_one_below) for measurement in FGFR21_distances_active]
FGFR22_distances_active_below = [calc_distance_below(measurement, threshold_neighbor_one_below) for measurement in FGFR22_distances_active]
FGFR23_distances_active_below = [calc_distance_below(measurement, threshold_neighbor_one_below) for measurement in FGFR23_distances_active]
FGFR24_distances_active_below = [calc_distance_below(measurement, threshold_neighbor_one_below) for measurement in FGFR24_distances_active]
FGFR31_distances_active_below = [calc_distance_below(measurement, threshold_neighbor_one_below) for measurement in FGFR31_distances_active]
FGFR32_distances_active_below = [calc_distance_below(measurement, threshold_neighbor_one_below) for measurement in FGFR32_distances_active]
FGFR33_distances_active_below = [calc_distance_below(measurement, threshold_neighbor_one_below) for measurement in FGFR33_distances_active]
FGFR34_distances_active_below = [calc_distance_below(measurement, threshold_neighbor_one_below) for measurement in FGFR34_distances_active]
FGFR41_distances_active_below = [calc_distance_below(measurement, threshold_neighbor_one_below) for measurement in FGFR41_distances_active]
FGFR42_distances_active_below = [calc_distance_below(measurement, threshold_neighbor_one_below) for measurement in FGFR42_distances_active]
FGFR43_distances_active_below = [calc_distance_below(measurement, threshold_neighbor_one_below) for measurement in FGFR43_distances_active]
FGFR44_distances_active_below = [calc_distance_below(measurement, threshold_neighbor_one_below) for measurement in FGFR44_distances_active]

#### Plot the results

- Set the y-axis max-range for all plots [0,1].
- Choose if single measurements should be displayed by setting single_neighbor_one_below to True.

In [ ]:
y_axis_max_neighbor_one_below = 0.15  # between 0 and 1
y_distances_threshold = 0.15  # between 0 and 1
bin_size_one_threshold = 1  # bin size of distances below threshold plot
single_neighbor_one_below = False

In [ ]:
def plot_distances_below_mean(na_1X, na_2X, na_3X, na_4X, a_1X, a_2X, a_3X, a_4X,
             na_1X_error, na_2X_error, na_3X_error, na_4X_error, a_1X_error, a_2X_error, a_3X_error, a_4X_error,
            subplot_titles, y_axis_max=1):   
    x_descs = []
    for i in range(1,5):
        x_desc = [str(i) + str(j) for j in range(1,5)]
        x_descs.append(x_desc)
    
    fig = make_subplots(rows=2, cols=2, subplot_titles=subplot_titles)
    trace0na = go.Bar(x=x_descs[0], y=na_1X, error_y=dict(type="data", array=na_1X_error, visible=True),
                     marker_color=condition_colors[0], name=subplot_titles[0] +" "+condition_names[0])
    trace0a = go.Bar(x=x_descs[0], y=a_1X, error_y=dict(type="data", array=a_1X_error, visible=True),
                      marker_color=condition_colors[1], name=subplot_titles[0] +" "+condition_names[1])
    trace1na = go.Bar(x=x_descs[1], y=na_2X, error_y=dict(type="data", array=na_2X_error, visible=True),
                      marker_color=condition_colors[0], name=subplot_titles[1] +" "+condition_names[0])
    trace1a = go.Bar(x=x_descs[1], y=a_2X, error_y=dict(type="data", array=a_2X_error, visible=True),
                      marker_color=condition_colors[1], name=subplot_titles[1] +" "+condition_names[1])
    trace2na = go.Bar(x=x_descs[2], y=na_3X, error_y=dict(type="data", array=na_3X_error, visible=True),
                      marker_color=condition_colors[0], name=subplot_titles[2] +" "+condition_names[0])
    trace2a = go.Bar(x=x_descs[2], y=a_3X, error_y=dict(type="data", array=a_3X_error, visible=True),
                      marker_color=condition_colors[1], name=subplot_titles[2] +" "+condition_names[1])
    trace3na = go.Bar(x=x_descs[3], y=na_4X, error_y=dict(type="data", array=na_4X_error, visible=True),
                      marker_color=condition_colors[0], name=subplot_titles[3] +" "+condition_names[0])
    trace3a = go.Bar(x=x_descs[3], y=a_4X, error_y=dict(type="data", array=a_4X_error, visible=True),
                      marker_color=condition_colors[1], name=subplot_titles[3] +" "+condition_names[1])
    fig.append_trace(trace0na, row=1, col=1)
    fig.append_trace(trace0a, row=1, col=1)
    fig.append_trace(trace1na, row=1, col=2)
    fig.append_trace(trace1a, row=1, col=2)
    fig.append_trace(trace2na, row=2, col=1)
    fig.append_trace(trace2a, row=2, col=1)
    fig.append_trace(trace3na, row=2, col=2)
    fig.append_trace(trace3a, row=2, col=2)
    fig.update_layout(title_text="Amount of nearest neighbor distances less or equal to {} nm, all measurements".format(threshold_neighbor_one_below),
                     barmode="group", template=background_template)
    fig.update_yaxes(title_text="fraction", range=[0,y_axis_max])
    fig.show()

def plot_distances_below_single(fgfr_11, fgfr_12, fgfr_13, fgfr_14, fgfr_21, fgfr_22, fgfr_23, fgfr_24,
                                fgfr_31, fgfr_32, fgfr_33, fgfr_34, fgfr_41, fgfr_42, fgfr_43, fgfr_44,
                                measurement_id, subplot_titles, active=False, y_axis_max=1):    
    fig = make_subplots(rows=2, cols=2, subplot_titles=subplot_titles,
                        specs=[[{"type": "bar"}, {"type": "bar"}], [{"type": "bar"}, {"type": "bar"}]])
        
    color = "Red" if active else "RoyalBlue"
    x_descs = []
    for j in range(1,5):
        x_desc = [str(j) + str(i) for i in range(1,5)]
        x_descs.append(x_desc)
        
    y_vals = []
    y_vals.append([fgfr_11, fgfr_12, fgfr_13, fgfr_14])
    y_vals.append([fgfr_21, fgfr_22, fgfr_23, fgfr_24])
    y_vals.append([fgfr_31, fgfr_32, fgfr_33, fgfr_34])
    y_vals.append([fgfr_41, fgfr_42, fgfr_43, fgfr_44])
    
    fig.add_trace(go.Bar(x=x_descs[0], y=y_vals[0], name=subplot_titles[0], marker=dict(color=color)), row=1, col=1)
    fig.add_trace(go.Bar(x=x_descs[1], y=y_vals[1], name=subplot_titles[1], marker=dict(color=color)), row=1, col=2)
    fig.add_trace(go.Bar(x=x_descs[2], y=y_vals[2], name=subplot_titles[2], marker=dict(color=color)), row=2, col=1)
    fig.add_trace(go.Bar(x=x_descs[3], y=y_vals[3], name=subplot_titles[3], marker=dict(color=color)), row=2, col=2)
    
    measurement_space = condition_names[1] if active else condition_names[0]
    fig.update_layout(title_text="Amount of nearest neighbor distances less or equal to {} nm, measurement {}, {}".format(threshold_neighbor_one_below, measurement_id, measurement_space),
                     barmode="group", template=background_template)
    fig.update_yaxes(title_text="fraction", range=[0,y_axis_max])
    fig.show()

subplot_titles_neighbor_one_below = [name+"->X" for name in target_names]

plot_distances_below_mean(distances_below_nactive_FGFR1X_mean, distances_below_nactive_FGFR2X_mean, distances_below_nactive_FGFR3X_mean, distances_below_nactive_FGFR4X_mean,
         distances_below_active_FGFR1X_mean, distances_below_active_FGFR2X_mean, distances_below_active_FGFR3X_mean, distances_below_active_FGFR4X_mean,
        distances_below_nactive_FGFR1X_mean_error, distances_below_nactive_FGFR2X_mean_error, distances_below_nactive_FGFR3X_mean_error, distances_below_nactive_FGFR4X_mean_error,
        distances_below_active_FGFR1X_mean_error, distances_below_active_FGFR2X_mean_error, distances_below_active_FGFR3X_mean_error, distances_below_active_FGFR4X_mean_error,
        subplot_titles=subplot_titles_neighbor_one_below, y_axis_max=y_axis_max_neighbor_one_below)

max_bin_neighbor_one_threshold = threshold_neighbor_one_below
bins_neighbor_one_threshold = [bin_size_one_threshold*i for i in range(max_bin_neighbor_one_threshold+1)]
bins_plot_neighbor_one_threshold = [bin_size_one_threshold*i for i in range(1,max_bin_neighbor_one_threshold+2)]

# all measurements not active 1->x
FGFR11_nactive_hist_threshold_mean, FGFR11_nactive_hist_threshold_mean_error = calc_mean_hist_values(distances_below_nactive_FGFR1X[0], bins_neighbor_one_threshold)
FGFR12_nactive_hist_threshold_mean, FGFR12_nactive_hist_threshold_mean_error = calc_mean_hist_values(distances_below_nactive_FGFR1X[1], bins_neighbor_one_threshold)
FGFR13_nactive_hist_threshold_mean, FGFR13_nactive_hist_threshold_mean_error = calc_mean_hist_values(distances_below_nactive_FGFR1X[2], bins_neighbor_one_threshold)
FGFR14_nactive_hist_threshold_mean, FGFR14_nactive_hist_threshold_mean_error = calc_mean_hist_values(distances_below_nactive_FGFR1X[3], bins_neighbor_one_threshold)
# all measurements not active 2->x
FGFR21_nactive_hist_threshold_mean, FGFR21_nactive_hist_threshold_mean_error = calc_mean_hist_values(distances_below_nactive_FGFR2X[0], bins_neighbor_one_threshold)
FGFR22_nactive_hist_threshold_mean, FGFR22_nactive_hist_threshold_mean_error = calc_mean_hist_values(distances_below_nactive_FGFR2X[1], bins_neighbor_one_threshold)
FGFR23_nactive_hist_threshold_mean, FGFR23_nactive_hist_threshold_mean_error = calc_mean_hist_values(distances_below_nactive_FGFR2X[2], bins_neighbor_one_threshold)
FGFR24_nactive_hist_threshold_mean, FGFR24_nactive_hist_threshold_mean_error = calc_mean_hist_values(distances_below_nactive_FGFR2X[3], bins_neighbor_one_threshold)
# all measurements not active 3->x
FGFR31_nactive_hist_threshold_mean, FGFR31_nactive_hist_threshold_mean_error = calc_mean_hist_values(distances_below_nactive_FGFR3X[0], bins_neighbor_one_threshold)
FGFR32_nactive_hist_threshold_mean, FGFR32_nactive_hist_threshold_mean_error = calc_mean_hist_values(distances_below_nactive_FGFR3X[1], bins_neighbor_one_threshold)
FGFR33_nactive_hist_threshold_mean, FGFR33_nactive_hist_threshold_mean_error = calc_mean_hist_values(distances_below_nactive_FGFR3X[2], bins_neighbor_one_threshold)
FGFR34_nactive_hist_threshold_mean, FGFR34_nactive_hist_threshold_mean_error = calc_mean_hist_values(distances_below_nactive_FGFR3X[3], bins_neighbor_one_threshold)
# all measurements not active 4->x
FGFR41_nactive_hist_threshold_mean, FGFR41_nactive_hist_threshold_mean_error = calc_mean_hist_values(distances_below_nactive_FGFR4X[0], bins_neighbor_one_threshold)
FGFR42_nactive_hist_threshold_mean, FGFR42_nactive_hist_threshold_mean_error = calc_mean_hist_values(distances_below_nactive_FGFR4X[1], bins_neighbor_one_threshold)
FGFR43_nactive_hist_threshold_mean, FGFR43_nactive_hist_threshold_mean_error = calc_mean_hist_values(distances_below_nactive_FGFR4X[2], bins_neighbor_one_threshold)
FGFR44_nactive_hist_threshold_mean, FGFR44_nactive_hist_threshold_mean_error = calc_mean_hist_values(distances_below_nactive_FGFR4X[3], bins_neighbor_one_threshold)
# all measurements active 1->x
FGFR11_active_hist_threshold_mean, FGFR11_active_hist_threshold_mean_error = calc_mean_hist_values(distances_below_active_FGFR1X[0], bins_neighbor_one_threshold)
FGFR12_active_hist_threshold_mean, FGFR12_active_hist_threshold_mean_error = calc_mean_hist_values(distances_below_active_FGFR1X[1], bins_neighbor_one_threshold)
FGFR13_active_hist_threshold_mean, FGFR13_active_hist_threshold_mean_error = calc_mean_hist_values(distances_below_active_FGFR1X[2], bins_neighbor_one_threshold)
FGFR14_active_hist_threshold_mean, FGFR14_active_hist_threshold_mean_error = calc_mean_hist_values(distances_below_active_FGFR1X[3], bins_neighbor_one_threshold)
# all measurements active 2->x
FGFR21_active_hist_threshold_mean, FGFR21_active_hist_threshold_mean_error = calc_mean_hist_values(distances_below_active_FGFR2X[0], bins_neighbor_one_threshold)
FGFR22_active_hist_threshold_mean, FGFR22_active_hist_threshold_mean_error = calc_mean_hist_values(distances_below_active_FGFR2X[1], bins_neighbor_one_threshold)
FGFR23_active_hist_threshold_mean, FGFR23_active_hist_threshold_mean_error = calc_mean_hist_values(distances_below_active_FGFR2X[2], bins_neighbor_one_threshold)
FGFR24_active_hist_threshold_mean, FGFR24_active_hist_threshold_mean_error = calc_mean_hist_values(distances_below_active_FGFR2X[3], bins_neighbor_one_threshold)
# all measurements active 3->x
FGFR31_active_hist_threshold_mean, FGFR31_active_hist_threshold_mean_error = calc_mean_hist_values(distances_below_active_FGFR3X[0], bins_neighbor_one_threshold)
FGFR32_active_hist_threshold_mean, FGFR32_active_hist_threshold_mean_error = calc_mean_hist_values(distances_below_active_FGFR3X[1], bins_neighbor_one_threshold)
FGFR33_active_hist_threshold_mean, FGFR33_active_hist_threshold_mean_error = calc_mean_hist_values(distances_below_active_FGFR3X[2], bins_neighbor_one_threshold)
FGFR34_active_hist_threshold_mean, FGFR34_active_hist_threshold_mean_error = calc_mean_hist_values(distances_below_active_FGFR3X[3], bins_neighbor_one_threshold)
# all measurements active 4->x
FGFR41_active_hist_threshold_mean, FGFR41_active_hist_threshold_mean_error = calc_mean_hist_values(distances_below_active_FGFR4X[0], bins_neighbor_one_threshold)
FGFR42_active_hist_threshold_mean, FGFR42_active_hist_threshold_mean_error = calc_mean_hist_values(distances_below_active_FGFR4X[1], bins_neighbor_one_threshold)
FGFR43_active_hist_threshold_mean, FGFR43_active_hist_threshold_mean_error = calc_mean_hist_values(distances_below_active_FGFR4X[2], bins_neighbor_one_threshold)
FGFR44_active_hist_threshold_mean, FGFR44_active_hist_threshold_mean_error = calc_mean_hist_values(distances_below_active_FGFR4X[3], bins_neighbor_one_threshold)

print("Histograms of distances less or equal to {} nm, all measurements.".format(threshold_neighbor_one_below))
plot_hist_mean(FGFR11_nactive_hist_threshold_mean, FGFR12_nactive_hist_threshold_mean, FGFR13_nactive_hist_threshold_mean, FGFR14_nactive_hist_threshold_mean,
              FGFR11_nactive_hist_threshold_mean_error, FGFR12_nactive_hist_threshold_mean_error, FGFR13_nactive_hist_threshold_mean_error, FGFR14_nactive_hist_threshold_mean_error,
              FGFR11_active_hist_threshold_mean, FGFR12_active_hist_threshold_mean, FGFR13_active_hist_threshold_mean, FGFR14_active_hist_threshold_mean,
              FGFR11_active_hist_threshold_mean_error, FGFR12_active_hist_threshold_mean_error, FGFR13_active_hist_threshold_mean_error, FGFR14_active_hist_threshold_mean_error,
              bins_plot_neighbor_one_threshold, plot_descs[0], y_axes_display_range=y_distances_threshold, x_axes_display_range=max_bin_neighbor_one_threshold)  # max_distance, None, integer
    
plot_hist_mean(FGFR21_nactive_hist_threshold_mean, FGFR22_nactive_hist_threshold_mean, FGFR23_nactive_hist_threshold_mean, FGFR24_nactive_hist_threshold_mean,
              FGFR21_nactive_hist_threshold_mean_error, FGFR22_nactive_hist_threshold_mean_error, FGFR23_nactive_hist_threshold_mean_error, FGFR24_nactive_hist_threshold_mean_error,
              FGFR21_active_hist_threshold_mean, FGFR22_active_hist_threshold_mean, FGFR23_active_hist_threshold_mean, FGFR24_active_hist_threshold_mean,
              FGFR21_active_hist_threshold_mean_error, FGFR22_active_hist_threshold_mean_error, FGFR23_active_hist_threshold_mean_error, FGFR24_active_hist_threshold_mean_error,
              bins_plot_neighbor_one_threshold, plot_descs[1], y_axes_display_range=y_distances_threshold, x_axes_display_range=max_bin_neighbor_one_threshold)  # max_distance, None, integer

plot_hist_mean(FGFR31_nactive_hist_threshold_mean, FGFR32_nactive_hist_threshold_mean, FGFR33_nactive_hist_threshold_mean, FGFR34_nactive_hist_threshold_mean,
              FGFR31_nactive_hist_threshold_mean_error, FGFR32_nactive_hist_threshold_mean_error, FGFR33_nactive_hist_threshold_mean_error, FGFR34_nactive_hist_threshold_mean_error,
              FGFR31_active_hist_threshold_mean, FGFR32_active_hist_threshold_mean, FGFR33_active_hist_threshold_mean, FGFR34_active_hist_threshold_mean,
              FGFR31_active_hist_threshold_mean_error, FGFR32_active_hist_threshold_mean_error, FGFR33_active_hist_threshold_mean_error, FGFR34_active_hist_threshold_mean_error,
              bins_plot_neighbor_one_threshold, plot_descs[2], y_axes_display_range=y_distances_threshold, x_axes_display_range=max_bin_neighbor_one_threshold)  # max_distance, None, integer
    
plot_hist_mean(FGFR41_nactive_hist_threshold_mean, FGFR42_nactive_hist_threshold_mean, FGFR43_nactive_hist_threshold_mean, FGFR44_nactive_hist_threshold_mean,
              FGFR41_nactive_hist_threshold_mean_error, FGFR42_nactive_hist_threshold_mean_error, FGFR43_nactive_hist_threshold_mean_error, FGFR44_nactive_hist_threshold_mean_error,
              FGFR41_active_hist_threshold_mean, FGFR42_active_hist_threshold_mean, FGFR43_active_hist_threshold_mean, FGFR44_active_hist_threshold_mean,
              FGFR41_active_hist_threshold_mean_error, FGFR42_active_hist_threshold_mean_error, FGFR43_active_hist_threshold_mean_error, FGFR44_active_hist_threshold_mean_error,
              bins_plot_neighbor_one_threshold, plot_descs[3], y_axes_display_range=y_distances_threshold, x_axes_display_range=max_bin_neighbor_one_threshold)  # max_distance, None, integer


if single_neighbor_one_below:
    for i in range(len(FGFR11_distances_nactive_below)):
        plot_distances_below_single(FGFR11_distances_nactive_below[i][0], FGFR12_distances_nactive_below[i][0], FGFR13_distances_nactive_below[i][0], FGFR14_distances_nactive_below[i][0],
                                    FGFR21_distances_nactive_below[i][0], FGFR22_distances_nactive_below[i][0], FGFR23_distances_nactive_below[i][0], FGFR24_distances_nactive_below[i][0],
                                    FGFR31_distances_nactive_below[i][0], FGFR32_distances_nactive_below[i][0], FGFR33_distances_nactive_below[i][0], FGFR34_distances_nactive_below[i][0],
                                    FGFR41_distances_nactive_below[i][0], FGFR42_distances_nactive_below[i][0], FGFR43_distances_nactive_below[i][0], FGFR44_distances_nactive_below[i][0],
                                    i+1, subplot_titles_neighbor_one_below, active=False, y_axis_max=y_axis_max_neighbor_one_below)

    for i in range(len(FGFR11_distances_active_below)):
        plot_distances_below_single(FGFR11_distances_active_below[i][0], FGFR12_distances_active_below[i][0], FGFR13_distances_active_below[i][0], FGFR14_distances_active_below[i][0],
                                    FGFR21_distances_active_below[i][0], FGFR22_distances_active_below[i][0], FGFR23_distances_active_below[i][0], FGFR24_distances_active_below[i][0],
                                    FGFR31_distances_active_below[i][0], FGFR32_distances_active_below[i][0], FGFR33_distances_active_below[i][0], FGFR34_distances_active_below[i][0],
                                    FGFR41_distances_active_below[i][0], FGFR42_distances_active_below[i][0], FGFR43_distances_active_below[i][0], FGFR44_distances_active_below[i][0],
                                    i+1, subplot_titles_neighbor_one_below, active=True, y_axis_max=y_axis_max_neighbor_one_below)

## Nearest neighbor distances with all neighbor types

For each center determine the type and distance of the nearest neighbor. The neighbor space includes all targets. <br/>
FGFR1->X, FGFR2->X, FGFR3->X, FGFR4->X; X=1,2,3,4

#### Execute the analysis

In [ ]:
start = time.time()
print("Analyzing not active measurements ...")
FGFR1X_distances_nactive, FGFR2X_distances_nactive, FGFR3X_distances_nactive, FGFR4X_distances_nactive = [], [], [], [] 
FGFR1X_types_nactive, FGFR2X_types_nactive, FGFR3X_types_nactive, FGFR4X_types_nactive = [], [], [], []
with tqdm(total=len(FGFR1_not_active), desc="Measurement") as pbar:
    for fgfr1, fgfr2, fgfr3, fgfr4 in zip(FGFR1_not_active, FGFR2_not_active, FGFR3_not_active, FGFR4_not_active):
        with tqdm(total=4, desc="FGFR", leave=False) as pbar_B:
            FGFR1_nactive_xy = xy_stack(fgfr1["clusters"]["com_x"][:reduce_size], fgfr1["clusters"]["com_y"][:reduce_size], pixel_size=pixel_size)
            FGFR2_nactive_xy = xy_stack(fgfr2["clusters"]["com_x"][:reduce_size], fgfr2["clusters"]["com_y"][:reduce_size], pixel_size=pixel_size)
            FGFR3_nactive_xy = xy_stack(fgfr3["clusters"]["com_x"][:reduce_size], fgfr3["clusters"]["com_y"][:reduce_size], pixel_size=pixel_size)
            FGFR4_nactive_xy = xy_stack(fgfr4["clusters"]["com_x"][:reduce_size], fgfr4["clusters"]["com_y"][:reduce_size], pixel_size=pixel_size)
            neighbors_nactive_FGFR1234 = []
            neighbors_nactive_FGFR1234.append(FGFR1_nactive_xy[:reduce_size])
            neighbors_nactive_FGFR1234.append(FGFR2_nactive_xy[:reduce_size])
            neighbors_nactive_FGFR1234.append(FGFR3_nactive_xy[:reduce_size])
            neighbors_nactive_FGFR1234.append(FGFR4_nactive_xy[:reduce_size])
            number_subregions_nactive = int(np.floor(math.sqrt(sum([len(i) for i in neighbors_nactive_FGFR1234]))))
            grid_1X = GridNNSearch(FGFR1_nactive_xy[:reduce_size], neighbors_nactive_FGFR1234,
                                   number_subregions_nactive, n_pixels, pixel_size)
            nn_1X_distances, nn_1X_types = grid_1X.get_nn_distances()
            FGFR1X_distances_nactive.append(nn_1X_distances)
            FGFR1X_types_nactive.append(nn_1X_types)
            pbar_B.update(1)
            grid_2X = GridNNSearch(FGFR2_nactive_xy[:reduce_size], neighbors_nactive_FGFR1234,
                                   number_subregions_nactive, n_pixels, pixel_size)
            nn_2X_distances, nn_2X_types = grid_2X.get_nn_distances()
            FGFR2X_distances_nactive.append(nn_2X_distances)
            FGFR2X_types_nactive.append(nn_2X_types)
            pbar_B.update(1)
            grid_3X = GridNNSearch(FGFR3_nactive_xy[:reduce_size], neighbors_nactive_FGFR1234,
                                   number_subregions_nactive, n_pixels, pixel_size)
            nn_3X_distances, nn_3X_types = grid_3X.get_nn_distances()
            FGFR3X_distances_nactive.append(nn_3X_distances)
            FGFR3X_types_nactive.append(nn_3X_types)
            pbar_B.update(1)
            grid_4X = GridNNSearch(FGFR4_nactive_xy[:reduce_size], neighbors_nactive_FGFR1234,
                                   number_subregions_nactive, n_pixels, pixel_size)
            nn_4X_distances, nn_4X_types = grid_4X.get_nn_distances()
            FGFR4X_distances_nactive.append(nn_4X_distances)
            FGFR4X_types_nactive.append(nn_4X_types)
            pbar_B.update(1)
            pbar.update(1)
        
print("Analyzing active measurements ...")
FGFR1X_distances_active, FGFR2X_distances_active, FGFR3X_distances_active, FGFR4X_distances_active = [], [], [], [] 
FGFR1X_types_active, FGFR2X_types_active, FGFR3X_types_active, FGFR4X_types_active = [], [], [], []
with tqdm(total=len(FGFR1_active), desc="Measurement") as pbar:
    for fgfr1, fgfr2, fgfr3, fgfr4 in zip(FGFR1_active, FGFR2_active, FGFR3_active, FGFR4_active):
        with tqdm(total=4, desc="FGFR", leave=False) as pbar_B:
            FGFR1_active_xy = xy_stack(fgfr1["clusters"]["com_x"][:reduce_size], fgfr1["clusters"]["com_y"][:reduce_size], pixel_size=pixel_size)
            FGFR2_active_xy = xy_stack(fgfr2["clusters"]["com_x"][:reduce_size], fgfr2["clusters"]["com_y"][:reduce_size], pixel_size=pixel_size)
            FGFR3_active_xy = xy_stack(fgfr3["clusters"]["com_x"][:reduce_size], fgfr3["clusters"]["com_y"][:reduce_size], pixel_size=pixel_size)
            FGFR4_active_xy = xy_stack(fgfr4["clusters"]["com_x"][:reduce_size], fgfr4["clusters"]["com_y"][:reduce_size], pixel_size=pixel_size)
            neighbors_active_FGFR1234 = []
            neighbors_active_FGFR1234.append(FGFR1_active_xy[:reduce_size])
            neighbors_active_FGFR1234.append(FGFR2_active_xy[:reduce_size])
            neighbors_active_FGFR1234.append(FGFR3_active_xy[:reduce_size])
            neighbors_active_FGFR1234.append(FGFR4_active_xy[:reduce_size])
            number_subregions_active = int(np.floor(math.sqrt(sum([len(i) for i in neighbors_active_FGFR1234]))))
            grid_1X = GridNNSearch(FGFR1_active_xy[:reduce_size], neighbors_active_FGFR1234,
                                   number_subregions_active, n_pixels, pixel_size)
            nn_1X_distances, nn_1X_types = grid_1X.get_nn_distances()
            FGFR1X_distances_active.append(nn_1X_distances)
            FGFR1X_types_active.append(nn_1X_types)
            pbar_B.update(1)
            grid_2X = GridNNSearch(FGFR2_active_xy[:reduce_size], neighbors_active_FGFR1234,
                                   number_subregions_active, n_pixels, pixel_size)
            nn_2X_distances, nn_2X_types = grid_2X.get_nn_distances()
            FGFR2X_distances_active.append(nn_2X_distances)
            FGFR2X_types_active.append(nn_2X_types)
            pbar_B.update(1)
            grid_3X = GridNNSearch(FGFR3_active_xy[:reduce_size], neighbors_active_FGFR1234,
                                   number_subregions_active, n_pixels, pixel_size)
            nn_3X_distances, nn_3X_types = grid_3X.get_nn_distances()
            FGFR3X_distances_active.append(nn_3X_distances)
            FGFR3X_types_active.append(nn_3X_types)
            pbar_B.update(1)
            grid_4X = GridNNSearch(FGFR4_active_xy[:reduce_size], neighbors_active_FGFR1234,
                                   number_subregions_active, n_pixels, pixel_size)
            nn_4X_distances, nn_4X_types = grid_4X.get_nn_distances()
            FGFR4X_distances_active.append(nn_4X_distances)
            FGFR4X_types_active.append(nn_4X_types)
            pbar_B.update(1)
            pbar.update(1)            

# Calculate type counts, not active
FGFR1X_types_nactive_counts, FGFR2X_types_nactive_counts, FGFR3X_types_nactive_counts, FGFR4X_types_nactive_counts = [], [], [], [] 
for count, (na_1x, na_2x, na_3x, na_4x) in enumerate(zip(FGFR1X_types_nactive, FGFR2X_types_nactive, FGFR3X_types_nactive, FGFR4X_types_nactive), 1):
    na_1x_counts = [na_1x.count(i) for i in range(4)]
    na_2x_counts = [na_2x.count(i) for i in range(4)]
    na_3x_counts = [na_3x.count(i) for i in range(4)]
    na_4x_counts = [na_4x.count(i) for i in range(4)]
    na_1x_counts /= np.sum(na_1x_counts)
    na_2x_counts /= np.sum(na_2x_counts)
    na_3x_counts /= np.sum(na_3x_counts)
    na_4x_counts /= np.sum(na_4x_counts)
    FGFR1X_types_nactive_counts.append(na_1x_counts)
    FGFR2X_types_nactive_counts.append(na_2x_counts)
    FGFR3X_types_nactive_counts.append(na_3x_counts)
    FGFR4X_types_nactive_counts.append(na_4x_counts)

FGFR1X_types_nactive_counts_mean, FGFR1X_types_nactive_counts_mean_error = calc_mean_error(FGFR1X_types_nactive_counts)
FGFR2X_types_nactive_counts_mean, FGFR2X_types_nactive_counts_mean_error = calc_mean_error(FGFR2X_types_nactive_counts)
FGFR3X_types_nactive_counts_mean, FGFR3X_types_nactive_counts_mean_error = calc_mean_error(FGFR3X_types_nactive_counts)
FGFR4X_types_nactive_counts_mean, FGFR4X_types_nactive_counts_mean_error = calc_mean_error(FGFR4X_types_nactive_counts)

# Calculate type counts, active
FGFR1X_types_active_counts, FGFR2X_types_active_counts, FGFR3X_types_active_counts, FGFR4X_types_active_counts = [], [], [], [] 
for count, (a_1x, a_2x, a_3x, a_4x) in enumerate(zip(FGFR1X_types_active, FGFR2X_types_active, FGFR3X_types_active, FGFR4X_types_active), 1):
    a_1x_counts = [a_1x.count(i) for i in range(4)]
    a_2x_counts = [a_2x.count(i) for i in range(4)]
    a_3x_counts = [a_3x.count(i) for i in range(4)]
    a_4x_counts = [a_4x.count(i) for i in range(4)]
    a_1x_counts /= np.sum(a_1x_counts)
    a_2x_counts /= np.sum(a_2x_counts)
    a_3x_counts /= np.sum(a_3x_counts)
    a_4x_counts /= np.sum(a_4x_counts)
    FGFR1X_types_active_counts.append(a_1x_counts)
    FGFR2X_types_active_counts.append(a_2x_counts)
    FGFR3X_types_active_counts.append(a_3x_counts)
    FGFR4X_types_active_counts.append(a_4x_counts)

FGFR1X_types_active_counts_mean, FGFR1X_types_active_counts_mean_error = calc_mean_error(FGFR1X_types_active_counts)
FGFR2X_types_active_counts_mean, FGFR2X_types_active_counts_mean_error = calc_mean_error(FGFR2X_types_active_counts)
FGFR3X_types_active_counts_mean, FGFR3X_types_active_counts_mean_error = calc_mean_error(FGFR3X_types_active_counts)
FGFR4X_types_active_counts_mean, FGFR4X_types_active_counts_mean_error = calc_mean_error(FGFR4X_types_active_counts)

FGFR1X_distances_nactive_all = [j for i in FGFR1X_distances_nactive for j in i]
FGFR2X_distances_nactive_all = [j for i in FGFR2X_distances_nactive for j in i]
FGFR3X_distances_nactive_all = [j for i in FGFR3X_distances_nactive for j in i]
FGFR4X_distances_nactive_all = [j for i in FGFR4X_distances_nactive for j in i]
FGFR1X_distances_active_all = [j for i in FGFR1X_distances_active for j in i]
FGFR2X_distances_active_all = [j for i in FGFR2X_distances_active for j in i]
FGFR3X_distances_active_all = [j for i in FGFR3X_distances_active for j in i]
FGFR4X_distances_active_all = [j for i in FGFR4X_distances_active for j in i]

print("Total Execution took {0:.2f} s.".format(time.time()-start))

#### Plot the results

- The neighbor space is not resticted to one receptor type, all receptor types are considered.

##### Distances
- Choose a bin size.
- Set single_neighbor_all to True if single measurements should be displayed as histograms.
- Set the x-axis range with the x_axes_display_range variable: max_distance, None, integer, max_distance_mi (for single measurements).
- Set the y-axis range with "max" (max y-value), float (e.g. 0.4) or None (individual max per plot).

In [ ]:
bin_size_neighbor_all = 20
single_neighbor_all = True
y_max_value_all = 0.22 # "max" or float

In [ ]:
max_distance_neighbor_all = max(FGFR1X_distances_active_all+FGFR2X_distances_active_all+FGFR3X_distances_active_all+FGFR4X_distances_active_all
                  +FGFR1X_distances_nactive_all+FGFR2X_distances_nactive_all+FGFR3X_distances_nactive_all+FGFR4X_distances_nactive_all)

max_bin_neighbor_all = int(np.ceil(max_distance_neighbor_all/bin_size_neighbor_all))
bins_neighbor_all = [bin_size_neighbor_all*i for i in range(max_bin_neighbor_all+1)]
bins_plot_neighbor_all = [bin_size_neighbor_all*i for i in range(1,max_bin_neighbor_all+2)]

FGFR1X_nactive_hist_mean, FGFR1X_nactive_hist_mean_error = calc_mean_hist_values(FGFR1X_distances_nactive, bins_neighbor_all)
FGFR2X_nactive_hist_mean, FGFR2X_nactive_hist_mean_error = calc_mean_hist_values(FGFR2X_distances_nactive, bins_neighbor_all)
FGFR3X_nactive_hist_mean, FGFR3X_nactive_hist_mean_error = calc_mean_hist_values(FGFR3X_distances_nactive, bins_neighbor_all)
FGFR4X_nactive_hist_mean, FGFR4X_nactive_hist_mean_error = calc_mean_hist_values(FGFR4X_distances_nactive, bins_neighbor_all)
FGFR1X_active_hist_mean, FGFR1X_active_hist_mean_error = calc_mean_hist_values(FGFR1X_distances_active, bins_neighbor_all)
FGFR2X_active_hist_mean, FGFR2X_active_hist_mean_error = calc_mean_hist_values(FGFR2X_distances_active, bins_neighbor_all)
FGFR3X_active_hist_mean, FGFR3X_active_hist_mean_error = calc_mean_hist_values(FGFR3X_distances_active, bins_neighbor_all)
FGFR4X_active_hist_mean, FGFR4X_active_hist_mean_error = calc_mean_hist_values(FGFR4X_distances_active, bins_neighbor_all)

subplot_titles_neighbor_all = [name + "->X" for name in target_names]

# get y-axis max value
if y_max_value_all == "max":
    y_max_nn_all = max(np.append(FGFR1X_nactive_hist_mean, [FGFR2X_nactive_hist_mean, FGFR3X_nactive_hist_mean,
                                                           FGFR4X_nactive_hist_mean, FGFR1X_active_hist_mean,
                                                           FGFR2X_active_hist_mean, FGFR3X_active_hist_mean,
                                                           FGFR4X_active_hist_mean], axis=None))
    y_max_nn_all += 0.1
else: 
    y_max_nn_all = y_max_value_all
    
plot_hist_mean(FGFR1X_nactive_hist_mean, FGFR2X_nactive_hist_mean, FGFR3X_nactive_hist_mean, FGFR4X_nactive_hist_mean,
              FGFR1X_nactive_hist_mean_error, FGFR2X_nactive_hist_mean_error, FGFR3X_nactive_hist_mean_error, FGFR4X_nactive_hist_mean_error,
              FGFR1X_active_hist_mean, FGFR2X_active_hist_mean, FGFR3X_active_hist_mean, FGFR4X_active_hist_mean,
              FGFR1X_active_hist_mean_error, FGFR2X_active_hist_mean_error, FGFR3X_active_hist_mean_error, FGFR4X_active_hist_mean_error,
              bins_plot_neighbor_all, subplot_titles_neighbor_all, y_axes_display_range=y_max_nn_all, x_axes_display_range=400)  # max_distance, None, integer

if single_neighbor_all:
    for i, (na_1, na_2, na_3, na_4) in enumerate(zip(FGFR1X_distances_nactive, FGFR2X_distances_nactive, FGFR3X_distances_nactive, FGFR4X_distances_nactive), 1):
        max_distance_mi = max(na_1+na_2+na_3+na_4)
        plot_hist(na_1, na_2, na_3, na_4, single=True, measurement_id = str(i), max_distance=max_distance_neighbor_all, active=False,
              subplot_titles=subplot_titles_neighbor_all, bin_size=bin_size_neighbor_all, y_axes_display_range=y_max_nn_all, x_axes_display_range=max_distance_mi)  # max_distance, None, integer, max_distance_mi

    for i, (a_1, a_2, a_3, a_4) in enumerate(zip(FGFR1X_distances_active, FGFR2X_distances_active, FGFR3X_distances_active, FGFR4X_distances_active), 1):
        max_distance_mi = max(a_1+a_2+a_3+a_4)
        plot_hist(a_1, a_2, a_3, a_4, single=True, measurement_id = str(i), max_distance=max_distance_neighbor_all, active=True,
              subplot_titles=subplot_titles_neighbor_all, bin_size=bin_size_neighbor_all, y_axes_display_range=y_max_nn_all, x_axes_display_range=max_distance_mi)  # max_distance, None, integer, max_distance_mi

##### Nearest neighbor types
- Per receptor type, the type of nearest neighbor is determined.
- The type of nearest neighbor is normalized (sum(1->1, 1->2, 1->3, 1->4) = 1).
- "All measurements" plots are the average of single measurements, with mean errors.
- Set single_neighbor_all_types to True if the results of single measurements should be displayed.
- The types are depicted as barplots and matrix plots. 
    - Choose a predefined color palette or define a list of colors between which to interpolate.
    - Choose a font color.
    - Choose the number of decimals displayed within the matrix plot.
    - Set display_barplots_neighbor_all to True if the information in the matrix plot should additionally be displayed as barplots.

In [ ]:
# print availabe names of color scales, check them out here: https://plot.ly/python/builtin-colorscales/
named_colorscales = px.colors.named_colorscales()
print("\n".join(wrap("".join('{:<12}'.format(c) for c in named_colorscales), 96)))

In [ ]:
display_barplots_neighbor_all = False  # weather or not to plot information additionally as barplots
# choose a predefined color scale
colorscale_neighbor_all = "Mint"
# or define a color scale: colorscale = ["#0010ff", "#00b7ff", "#00ff9f", "#08ff00", "#afff00", "#ffa700", "#ff0000"]
# choose a font color "white", "black" ...
font_color_neighbor_all = ["black"]
# number of decimal places after the comma that are displayed in the heatmap
number_decimals_neighbor_all = 4  
single_neighbor_all_types = False  # display single measurements

In [ ]:
def plot_nn_types(FGFR1, FGFR2, FGFR3, FGFR4, FGFR1_error=None, FGFR2_error=None, FGFR3_error=None, FGFR4_error=None,
                  single=False, measurement_id="", active=False,
                  colorscale="Viridis", zmax=1, font_colors=["black"], number_decimals=4):
    if not single:
        number_decimals = "." + str(number_decimals) + "f"
        annotation_text = []
        FGFR4_round = [format(i, number_decimals) + " \u00B1 " + format(j, number_decimals) for i, j in zip(FGFR4, FGFR4_error)]
        annotation_text.append(FGFR4_round)
        FGFR3_round = [format(i, number_decimals) + " \u00B1 " + format(j, number_decimals) for i, j in zip(FGFR3, FGFR3_error)]
        annotation_text.append(FGFR3_round)
        FGFR2_round = [format(i, number_decimals) + " \u00B1 " + format(j, number_decimals) for i, j in zip(FGFR2, FGFR2_error)]
        annotation_text.append(FGFR2_round)
        FGFR1_round = [format(i, number_decimals) + " \u00B1 " + format(j, number_decimals) for i, j in zip(FGFR1, FGFR1_error)]
        annotation_text.append(FGFR1_round)
    else:
        number_decimals = "." + str(number_decimals) + "f"
        annotation_text = []
        FGFR4_round = [format(i, number_decimals) for i in FGFR4]
        annotation_text.append(FGFR4_round)
        FGFR3_round = [format(i, number_decimals) for i in FGFR3]
        annotation_text.append(FGFR3_round)
        FGFR2_round = [format(i, number_decimals) for i in FGFR2]
        annotation_text.append(FGFR2_round)
        FGFR1_round = [format(i, number_decimals) for i in FGFR1]
        annotation_text.append(FGFR1_round)
    fig = go.Figure(data=ff.create_annotated_heatmap(z=[FGFR4, FGFR3, FGFR2, FGFR1],
                                    x=target_names,
                                    y=target_names[::-1],
                                    annotation_text=annotation_text,
                                    zmin=0,
                                    zmax=zmax,
                                    colorscale=colorscale,
                                    font_colors=font_colors,
                                    showscale=True, reversescale=True))
    measurement_space = "single measurement " + measurement_id if single else "all measurements"
    measurement_condition = condition_names[1] if active else condition_names[0]
    fig.update_layout(title_text = measurement_condition + " " + measurement_space, template=background_template)
    fig.show()
    if display_barplots_neighbor_all and not single:
        fig = go.Figure([go.Bar(x=target_names, y=FGFR1, error_y=dict(type="data", array=FGFR1_error, visible=True))])
        fig.update_layout(title_text = measurement_condition + " " + measurement_space + ", " + target_names[0] + "->X",
                         template=background_template)
        fig.show()
        fig = go.Figure([go.Bar(x=target_names, y=FGFR2, error_y=dict(type="data", array=FGFR2_error, visible=True))])
        fig.update_layout(title_text = measurement_condition + " " + measurement_space + ", " + target_names[1] + "->X",
                         template=background_template)
        fig.show()
        fig = go.Figure([go.Bar(x=target_names, y=FGFR3, error_y=dict(type="data", array=FGFR3_error, visible=True))])
        fig.update_layout(title_text = measurement_condition + " " + measurement_space + ", " + target_names[2] + "->X",
                         template=background_template)
        fig.show()
        fig = go.Figure([go.Bar(x=target_names, y=FGFR4, error_y=dict(type="data", array=FGFR4_error, visible=True))])
        fig.update_layout(title_text = measurement_condition + " " + measurement_space + ", " + target_names[3] + "->X",
                         template=background_template)
        fig.show()
    # single measurements have no errors
    if display_barplots_neighbor_all and single:
        fig = go.Figure([go.Bar(x=target_names, y=FGFR1)])
        fig.update_layout(title_text = measurement_condition + " " + measurement_space + ", " + target_names[0] + "->X",
                         template=background_template)
        fig.show()
        fig = go.Figure([go.Bar(x=target_names, y=FGFR2)])
        fig.update_layout(title_text = measurement_condition + " " + measurement_space + ", " + target_names[1] + "->X",
                         template=background_template)
        fig.show()
        fig = go.Figure([go.Bar(x=target_names, y=FGFR3)])
        fig.update_layout(title_text = measurement_condition + " " + measurement_space + ", " + target_names[2] + "->X",
                         template=background_template)
        fig.show()
        fig = go.Figure([go.Bar(x=target_names, y=FGFR4)])
        fig.update_layout(title_text = measurement_condition + " " + measurement_space + ", " + target_names[3] + "->X",
                         template=background_template)
        fig.show()
        
zmax = max([j for i in FGFR1X_types_nactive_counts for j in i]+[j for i in FGFR2X_types_nactive_counts for j in i]
           +[j for i in FGFR3X_types_nactive_counts for j in i]+[j for i in FGFR4X_types_nactive_counts for j in i])
    
# mean not active measurements
plot_nn_types(FGFR1X_types_nactive_counts_mean, FGFR2X_types_nactive_counts_mean,
              FGFR3X_types_nactive_counts_mean, FGFR4X_types_nactive_counts_mean,
              FGFR1X_types_nactive_counts_mean_error, FGFR2X_types_nactive_counts_mean_error,
              FGFR3X_types_nactive_counts_mean_error, FGFR4X_types_nactive_counts_mean_error,
              single=False, active=False, colorscale=colorscale_neighbor_all, zmax=zmax,
              font_colors=font_color_neighbor_all, number_decimals=number_decimals_neighbor_all)
if single_neighbor_all_types:
    # single not active measurements
    for count, (na_1x, na_2x, na_3x, na_4x) in enumerate(zip(FGFR1X_types_nactive_counts, FGFR2X_types_nactive_counts, FGFR3X_types_nactive_counts, FGFR4X_types_nactive_counts), 1):
        plot_nn_types(na_1x, na_2x, na_3x, na_4x,
              single=True, measurement_id=str(count), active=False, colorscale=colorscale_neighbor_all, zmax=zmax,
                     font_colors=font_color_neighbor_all, number_decimals=number_decimals_neighbor_all)
        
# mean active measurements
plot_nn_types(FGFR1X_types_active_counts_mean, FGFR2X_types_active_counts_mean,
              FGFR3X_types_active_counts_mean, FGFR4X_types_active_counts_mean,
              FGFR1X_types_active_counts_mean_error, FGFR2X_types_active_counts_mean_error,
              FGFR3X_types_active_counts_mean_error, FGFR4X_types_active_counts_mean_error,
              single=False, active=True, colorscale=colorscale_neighbor_all, zmax=zmax,
             font_colors=font_color_neighbor_all, number_decimals=number_decimals_neighbor_all)
if single_neighbor_all_types:
    # single not active measurements
    for count, (a_1x, a_2x, a_3x, a_4x) in enumerate(zip(FGFR1X_types_active_counts, FGFR2X_types_active_counts, FGFR3X_types_active_counts, FGFR4X_types_active_counts), 1):
        plot_nn_types(a_1x, a_2x, a_3x, a_4x,
              single=True, measurement_id=str(count), active=True, colorscale=colorscale_neighbor_all, zmax=zmax,
                     font_colors=font_color_neighbor_all, number_decimals=number_decimals_neighbor_all)

### Amount of nearest neighbor distances less or equal to a threshold

- Set a threshold distance in nm, all distances <= threshold are accepted. The fraction of accepted distances is calculated and displayed as a barplots.
- Set the y-axis range of the barplots.
- Set single_neighbor_all_below to True if single measurements should be displayed.

In [ ]:
threshold_all_distances_below = 50  # accept distances <= threshold
y_axis_max_all_below = 0.35  # between 0 and 1 
single_neighbor_all_below = True

In [ ]:
def plot_distances_below_mean(FGFR_distances_nactive_mean, FGFR_distances_active_mean,
                             FGFR_distances_nactive_mean_error, FGFR_distances_active_mean_error,
                             y_axis_max=y_axis_max_all_below):
    x_desc = [target_names[0]+"->X", target_names[1]+"->X", target_names[2]+"->X", target_names[3]+"->X"]
    fig = go.Figure(data=[go.Bar(name=condition_names[0], x=x_desc, y=FGFR_distances_nactive_mean,
                                 marker=dict(color=condition_colors[0]),
                                error_y=dict(type="data", array=FGFR_distances_nactive_mean_error, visible=True)),
                         go.Bar(name=condition_names[1], x=x_desc, y=FGFR_distances_active_mean,
                                marker=dict(color=condition_colors[1]),
                                error_y=dict(type="data", array=FGFR_distances_active_mean_error, visible=True))])  
    fig.update_layout(title_text="Amount of nearest neighbor distances less or equal to {} nm, all measurements".format(threshold_all_distances_below),
                     barmode="group", template=background_template)
    fig.update_yaxes(title_text="fraction", range=[0,y_axis_max])
    fig.show()

FGFR_distances_nactive_mean, FGFR_distances_nactive_mean_error, _ = get_distance_below_fraction(FGFR1X_distances_nactive, FGFR2X_distances_nactive, FGFR3X_distances_nactive, FGFR4X_distances_nactive, threshold_all_distances_below)
FGFR_distances_active_mean, FGFR_distances_active_mean_error, _ = get_distance_below_fraction(FGFR1X_distances_active, FGFR2X_distances_active, FGFR3X_distances_active, FGFR4X_distances_active, threshold_all_distances_below)


plot_distances_below_mean(FGFR_distances_nactive_mean, FGFR_distances_active_mean,
                             FGFR_distances_nactive_mean_error, FGFR_distances_active_mean_error, y_axis_max_all_below)

FGFR1X_distances_nactive_below = [calc_distance_below(measurement, threshold_all_distances_below)[0] for measurement in FGFR1X_distances_nactive]
FGFR2X_distances_nactive_below = [calc_distance_below(measurement, threshold_all_distances_below)[0] for measurement in FGFR2X_distances_nactive]
FGFR3X_distances_nactive_below = [calc_distance_below(measurement, threshold_all_distances_below)[0] for measurement in FGFR3X_distances_nactive]
FGFR4X_distances_nactive_below = [calc_distance_below(measurement, threshold_all_distances_below)[0] for measurement in FGFR4X_distances_nactive]
FGFR1X_distances_active_below = [calc_distance_below(measurement, threshold_all_distances_below)[0] for measurement in FGFR1X_distances_active]
FGFR2X_distances_active_below = [calc_distance_below(measurement, threshold_all_distances_below)[0] for measurement in FGFR2X_distances_active]
FGFR3X_distances_active_below = [calc_distance_below(measurement, threshold_all_distances_below)[0] for measurement in FGFR3X_distances_active]
FGFR4X_distances_active_below = [calc_distance_below(measurement, threshold_all_distances_below)[0] for measurement in FGFR4X_distances_active]

# plot per single measurement
if single_neighbor_all_below:
    fig = make_subplots(rows=1, cols=2, subplot_titles=("not active", "active"), specs=[[{"type": "bar"}, {"type": "bar"}]])
    measurements_nactive = ["measurement " + str(i+1) for i in range(len(FGFR1X_distances_nactive_below))]
    measurements_active = ["measurement " + str(i+1) for i in range(len(FGFR1X_distances_active_below))]
    fig.add_trace(go.Bar(x=measurements_nactive, y=FGFR1X_distances_nactive_below, name=target_names[0]+"X "+condition_names[0], marker=dict(color=target_colors[0])), row=1, col=1)
    fig.add_trace(go.Bar(x=measurements_nactive, y=FGFR2X_distances_nactive_below, name=target_names[1]+"X "+condition_names[0], marker=dict(color=target_colors[1])), row=1, col=1)
    fig.add_trace(go.Bar(x=measurements_nactive, y=FGFR3X_distances_nactive_below, name=target_names[2]+"X "+condition_names[0], marker=dict(color=target_colors[2])), row=1, col=1)
    fig.add_trace(go.Bar(x=measurements_nactive, y=FGFR4X_distances_nactive_below, name=target_names[3]+"X "+condition_names[0], marker=dict(color=target_colors[3])), row=1, col=1)

    fig.add_trace(go.Bar(x=measurements_active, y=FGFR1X_distances_active_below, name=target_names[0]+"X "+condition_names[1], marker=dict(color=target_colors[0])), row=1, col=2)
    fig.add_trace(go.Bar(x=measurements_active, y=FGFR2X_distances_active_below, name=target_names[1]+"X "+condition_names[1], marker=dict(color=target_colors[1])), row=1, col=2)
    fig.add_trace(go.Bar(x=measurements_active, y=FGFR3X_distances_active_below, name=target_names[2]+"X "+condition_names[1], marker=dict(color=target_colors[2])), row=1, col=2)
    fig.add_trace(go.Bar(x=measurements_active, y=FGFR4X_distances_active_below, name=target_names[3]+"X "+condition_names[1], marker=dict(color=target_colors[3])), row=1, col=2)

    fig.update_layout(title_text="Amount of nearest neighbor distances less or equal to {} nm, per measurement".format(threshold_all_distances_below),
                     barmode="group", template=background_template)
    fig.update_yaxes(title_text="fraction", range=[0,y_axis_max_all_below], row=1, col=1)
    fig.update_yaxes(title_text="fraction", range=[0,y_axis_max_all_below], row=1, col=2)

    fig.show()

## Save results as hdf5

- Enter a path to save the file, the path includes the file name & .h5 ending, this overrides with no warning!

In [ ]:
save_path = r"C:\path_to_save\saved_file.h5" # path of hdf5 file with .h5 ending

In [ ]:
h5 = h5py.File(save_path, "w")
grp_number_to_paths = h5.create_group("measurement_id_path")

grp_one = h5.create_group("neighbors_one")
subgrp_one_distances = grp_one.create_group("distances")
subsubgrp_one_distances_mean = subgrp_one_distances.create_group("mean")
subsubgrp_one_distances_single = subgrp_one_distances.create_group("single")
subsubsubgrp_distances_mean_nactive = subsubgrp_one_distances_mean.create_group(condition_names[0])
subsubsubgrp_distances_mean_active = subsubgrp_one_distances_mean.create_group(condition_names[1])
subsubsubgrp_distances_single_nactive = subsubgrp_one_distances_single.create_group(condition_names[0])
subsubsubgrp_distances_single_active = subsubgrp_one_distances_single.create_group(condition_names[1])
subgrp_one_below = grp_one.create_group("distances_below")
subsubgrp_one_below_mean = subgrp_one_below.create_group("mean")
subsubgrp_one_below_single = subgrp_one_below.create_group("single")
subsubsubgrp_one_below_mean_nactive = subsubgrp_one_below_mean.create_group(condition_names[0])
subsubsubgrp_one_below_mean_active = subsubgrp_one_below_mean.create_group(condition_names[1])
subsubsubgrp_one_below_single_nactive = subsubgrp_one_below_single.create_group(condition_names[0])
subsubsubgrp_one_below_single_active = subsubgrp_one_below_single.create_group(condition_names[1])

grp_all = h5.create_group("neighbors_all")
subgrp_all_distances = grp_all.create_group("distances")
subsubgrp_distances_all_mean = subgrp_all_distances.create_group("mean")
subsubgrp_distances_all_single = subgrp_all_distances.create_group("single")
subsubsubgrp_distances_all_single_nactive = subsubgrp_distances_all_single.create_group(condition_names[0])
subsubsubgrp_distances_all_single_active = subsubgrp_distances_all_single.create_group(condition_names[1])
subgrp_all_types = grp_all.create_group("types")
subsubgrp_types_all_mean = subgrp_all_types.create_group("mean")
subsubgrp_types_all_single = subgrp_all_types.create_group("single")
subsubsubgrp_types_all_mean_nactive = subsubgrp_types_all_mean.create_group(condition_names[0])
subsubsubgrp_types_all_mean_active = subsubgrp_types_all_mean.create_group(condition_names[1])
subsubsubgrp_types_all_single_nactive = subsubgrp_types_all_single.create_group(condition_names[0])
subsubsubgrp_types_all_single_active = subsubgrp_types_all_single.create_group(condition_names[1])
subgrp_all_below = grp_all.create_group("distances_below")
subsubgrp_all_below_mean = subgrp_all_below.create_group("mean")
subsubgrp_all_below_single = subgrp_all_below.create_group("single")
subsubsubgrp_all_below_single_nactive = subsubgrp_all_below_single.create_group(condition_names[0])
subsubsubgrp_all_below_single_active = subsubgrp_all_below_single.create_group(condition_names[1])

# "measurement_id_path"
not_active_paths = [[a, b, c, d] for a, b, c, d in zip(not_active_FGFR1_paths, not_active_FGFR2_paths, not_active_FGFR3_paths, not_active_FGFR4_paths)]
not_active_paths = np.array(not_active_paths)
dset = grp_number_to_paths.create_dataset(condition_names[0], (len(not_active_FGFR1_paths),4), dtype=h5py.special_dtype(vlen=str))
dset[...] = not_active_paths
grp_number_to_paths[condition_names[0]].attrs["column_names"] = target_names
active_paths = [[a, b, c, d] for a, b, c, d in zip(active_FGFR1_paths, active_FGFR2_paths, active_FGFR3_paths, active_FGFR4_paths)]
active_paths = np.array(active_paths)
dset = grp_number_to_paths.create_dataset(condition_names[1], (len(active_FGFR1_paths),4), dtype=h5py.special_dtype(vlen=str))
dset[...] = active_paths
grp_number_to_paths[condition_names[1]].attrs["column_names"] = target_names

# distances mean, bin size
dset = subsubgrp_one_distances_mean.create_dataset("bin_size", (1,1), dtype=np.dtype([("bin size", int)]))
dset["bin size"] = bin_size_neighbor_one
# distances mean, not active
dset = subsubsubgrp_distances_mean_nactive.create_dataset(target_names[0]+"->X", (len(FGFR11_nactive_hist_mean),),
                                                      dtype=np.dtype([("distance", float),
                                                                    ("11" , float),
                                                                    ("\u0394 11", float),
                                                                    ("12", float),
                                                                    ("\u0394 12", float),
                                                                    ("13", float),
                                                                    ("\u0394 13", float),
                                                                    ("14", float),
                                                                    ("\u0394 14", float),]))
dset["distance"] = bins_plot_neighbor_one[:-1]
dset["11"] = FGFR11_nactive_hist_mean
dset["\u0394 11"] = FGFR11_nactive_hist_mean_error
dset["12"] = FGFR12_nactive_hist_mean
dset["\u0394 12"] = FGFR12_nactive_hist_mean_error
dset["13"] = FGFR13_nactive_hist_mean
dset["\u0394 13"] = FGFR13_nactive_hist_mean_error
dset["14"] = FGFR14_nactive_hist_mean
dset["\u0394 14"] = FGFR14_nactive_hist_mean_error
dset = subsubsubgrp_distances_mean_nactive.create_dataset(target_names[1]+"->X", (len(FGFR21_nactive_hist_mean),),
                                                      dtype=np.dtype([("distance", float),
                                                                    ("21" , float),
                                                                    ("\u0394 21", float),
                                                                    ("22", float),
                                                                    ("\u0394 22", float),
                                                                    ("23", float),
                                                                    ("\u0394 23", float),
                                                                    ("24", float),                                                                    
                                                                    ("\u0394 24", float)]))
dset["distance"] = bins_plot_neighbor_one[:-1]
dset["21"] = FGFR21_nactive_hist_mean
dset["\u0394 21"] = FGFR21_nactive_hist_mean_error
dset["22"] = FGFR22_nactive_hist_mean
dset["\u0394 22"] = FGFR22_nactive_hist_mean_error
dset["23"] = FGFR23_nactive_hist_mean
dset["\u0394 23"] = FGFR23_nactive_hist_mean_error
dset["24"] = FGFR24_nactive_hist_mean
dset["\u0394 24"] = FGFR24_nactive_hist_mean_error
dset = subsubsubgrp_distances_mean_nactive.create_dataset(target_names[2]+"->X", (len(FGFR31_nactive_hist_mean),),
                                                      dtype=np.dtype([("distance", float),
                                                                    ("31" , float),
                                                                    ("\u0394 31", float),
                                                                    ("32", float),
                                                                    ("\u0394 32", float),
                                                                    ("33", float),
                                                                    ("\u0394 33", float),
                                                                    ("34", float),
                                                                    ("\u0394 34", float)]))
dset["distance"] = bins_plot_neighbor_one[:-1]
dset["31"] = FGFR31_nactive_hist_mean
dset["\u0394 31"] = FGFR31_nactive_hist_mean_error
dset["32"] = FGFR32_nactive_hist_mean
dset["\u0394 32"] = FGFR32_nactive_hist_mean_error
dset["33"] = FGFR33_nactive_hist_mean
dset["\u0394 33"] = FGFR33_nactive_hist_mean_error
dset["34"] = FGFR34_nactive_hist_mean
dset["\u0394 34"] = FGFR34_nactive_hist_mean_error
dset = subsubsubgrp_distances_mean_nactive.create_dataset(target_names[3]+"->X", (len(FGFR41_nactive_hist_mean),),
                                                      dtype=np.dtype([("distance", float),
                                                                    ("41" , float),
                                                                    ("\u0394 41", float),
                                                                    ("42", float),
                                                                    ("\u0394 42", float),
                                                                    ("43", float),
                                                                    ("\u0394 43", float),
                                                                    ("44", float),
                                                                    ("\u0394 44", float)]))
dset["distance"] = bins_plot_neighbor_one[:-1]
dset["41"] = FGFR41_nactive_hist_mean
dset["\u0394 41"] = FGFR41_nactive_hist_mean_error
dset["42"] = FGFR42_nactive_hist_mean
dset["\u0394 42"] = FGFR42_nactive_hist_mean_error
dset["43"] = FGFR43_nactive_hist_mean
dset["\u0394 43"] = FGFR43_nactive_hist_mean_error
dset["44"] = FGFR44_nactive_hist_mean
dset["\u0394 44"] = FGFR44_nactive_hist_mean_error
# distances mean, active
dset = subsubsubgrp_distances_mean_active.create_dataset(target_names[0]+"->X", (len(FGFR11_active_hist_mean),),
                                                      dtype=np.dtype([("distance", float),
                                                                    ("11" , float),
                                                                    ("\u0394 11", float),
                                                                    ("12", float),
                                                                    ("\u0394 12", float),
                                                                    ("13", float),
                                                                    ("\u0394 13", float),
                                                                    ("14", float),
                                                                    ("\u0394 14", float),]))
dset["distance"] = bins_plot_neighbor_one[:-1]
dset["11"] = FGFR11_active_hist_mean
dset["\u0394 11"] = FGFR11_active_hist_mean_error
dset["12"] = FGFR12_active_hist_mean
dset["\u0394 12"] = FGFR12_active_hist_mean_error
dset["13"] = FGFR13_active_hist_mean
dset["\u0394 13"] = FGFR13_active_hist_mean_error
dset["14"] = FGFR14_active_hist_mean
dset["\u0394 14"] = FGFR14_active_hist_mean_error
dset = subsubsubgrp_distances_mean_active.create_dataset(target_names[1]+"->X", (len(FGFR21_active_hist_mean),),
                                                      dtype=np.dtype([("distance", float),
                                                                    ("21" , float),
                                                                    ("\u0394 21", float),
                                                                    ("22", float),
                                                                    ("\u0394 22", float),
                                                                    ("23", float),
                                                                    ("\u0394 23", float),
                                                                    ("24", float),                                                                    
                                                                    ("\u0394 24", float)]))
dset["distance"] = bins_plot_neighbor_one[:-1]
dset["21"] = FGFR21_active_hist_mean
dset["\u0394 21"] = FGFR21_active_hist_mean_error
dset["22"] = FGFR22_active_hist_mean
dset["\u0394 22"] = FGFR22_active_hist_mean_error
dset["23"] = FGFR23_active_hist_mean
dset["\u0394 23"] = FGFR23_active_hist_mean_error
dset["24"] = FGFR24_active_hist_mean
dset["\u0394 24"] = FGFR24_active_hist_mean_error
dset = subsubsubgrp_distances_mean_active.create_dataset(target_names[2]+"->X", (len(FGFR31_active_hist_mean),),
                                                      dtype=np.dtype([("distance", float),
                                                                    ("31" , float),
                                                                    ("\u0394 31", float),
                                                                    ("32", float),
                                                                    ("\u0394 32", float),
                                                                    ("33", float),
                                                                    ("\u0394 33", float),
                                                                    ("34", float),
                                                                    ("\u0394 34", float)]))
dset["distance"] = bins_plot_neighbor_one[:-1]
dset["31"] = FGFR31_active_hist_mean
dset["\u0394 31"] = FGFR31_active_hist_mean_error
dset["32"] = FGFR32_active_hist_mean
dset["\u0394 32"] = FGFR32_active_hist_mean_error
dset["33"] = FGFR33_active_hist_mean
dset["\u0394 33"] = FGFR33_active_hist_mean_error
dset["34"] = FGFR34_active_hist_mean
dset["\u0394 34"] = FGFR34_active_hist_mean_error
dset = subsubsubgrp_distances_mean_active.create_dataset(target_names[3]+"->X", (len(FGFR41_active_hist_mean),),
                                                      dtype=np.dtype([("distance", float),
                                                                    ("41" , float),
                                                                    ("\u0394 41", float),
                                                                    ("42", float),
                                                                    ("\u0394 42", float),
                                                                    ("43", float),
                                                                    ("\u0394 43", float),
                                                                    ("44", float),
                                                                    ("\u0394 44", float)]))
dset["distance"] = bins_plot_neighbor_one[:-1]
dset["41"] = FGFR41_active_hist_mean
dset["\u0394 41"] = FGFR41_active_hist_mean_error
dset["42"] = FGFR42_active_hist_mean
dset["\u0394 42"] = FGFR42_active_hist_mean_error
dset["43"] = FGFR43_active_hist_mean
dset["\u0394 43"] = FGFR43_active_hist_mean_error
dset["44"] = FGFR44_active_hist_mean
dset["\u0394 44"] = FGFR44_active_hist_mean_error



# distances single, not active
for i, (fgfr1, fgfr2, fgfr3, fgfr4) in enumerate(zip(FGFR1_nn_distances_nactive, FGFR2_nn_distances_nactive, FGFR3_nn_distances_nactive, FGFR4_nn_distances_nactive), 1):
    i = str(i) if i > 9 else "0" + str(i)
    measurement_subgrp = subsubsubgrp_distances_single_nactive.create_group("measurement_" + i)
    dset = measurement_subgrp.create_dataset(target_names[0]+"->X", (len(fgfr1[0]),),
                                                      dtype=np.dtype([("11" , float),
                                                                    ("12", float),
                                                                    ("13", float),
                                                                    ("14", float)]))
    dset["11"], dset["12"], dset["13"], dset["14"] = fgfr1[0], fgfr1[1], fgfr1[2], fgfr1[3]
    dset = measurement_subgrp.create_dataset(target_names[1]+"->X", (len(fgfr2[0]),),
                                                      dtype=np.dtype([("21" , float),
                                                                    ("22", float),
                                                                    ("23", float),
                                                                    ("24", float)]))
    dset["21"], dset["22"], dset["23"], dset["24"] = fgfr2[0], fgfr2[1], fgfr2[2], fgfr2[3]
    dset = measurement_subgrp.create_dataset(target_names[2]+"->X", (len(fgfr3[0]),),
                                                      dtype=np.dtype([("31" , float),
                                                                    ("32", float),
                                                                    ("33", float),
                                                                    ("34", float)]))
    dset["31"], dset["32"], dset["33"], dset["34"] = fgfr3[0], fgfr3[1], fgfr3[2], fgfr3[3]
    dset = measurement_subgrp.create_dataset(target_names[3]+"->X", (len(fgfr4[0]),),
                                                      dtype=np.dtype([("41" , float),
                                                                    ("42", float),
                                                                    ("43", float),
                                                                    ("44", float)]))
    dset["41"], dset["42"], dset["43"], dset["44"] = fgfr4[0], fgfr4[1], fgfr4[2], fgfr4[3]
# distances single, active
for i, (fgfr1, fgfr2, fgfr3, fgfr4) in enumerate(zip(FGFR1_nn_distances_active, FGFR2_nn_distances_active, FGFR3_nn_distances_active, FGFR4_nn_distances_active), 1):
    i = str(i) if i > 9 else "0" + str(i)
    measurement_subgrp = subsubsubgrp_distances_single_active.create_group("measurement_" + i)
    dset = measurement_subgrp.create_dataset(target_names[0]+"->X", (len(fgfr1[0]),),
                                                      dtype=np.dtype([("11" , float),
                                                                    ("12", float),
                                                                    ("13", float),
                                                                    ("14", float)]))
    dset["11"], dset["12"], dset["13"], dset["14"] = fgfr1[0], fgfr1[1], fgfr1[2], fgfr1[3]
    dset = measurement_subgrp.create_dataset(target_names[1]+"->X", (len(fgfr2[0]),),
                                                      dtype=np.dtype([("21" , float),
                                                                    ("22", float),
                                                                    ("23", float),
                                                                    ("24", float)]))
    dset["21"], dset["22"], dset["23"], dset["24"] = fgfr2[0], fgfr2[1], fgfr2[2], fgfr2[3]
    dset = measurement_subgrp.create_dataset(target_names[2]+"->X", (len(fgfr3[0]),),
                                                      dtype=np.dtype([("31" , float),
                                                                    ("32", float),
                                                                    ("33", float),
                                                                    ("34", float)]))
    dset["31"], dset["32"], dset["33"], dset["34"] = fgfr3[0], fgfr3[1], fgfr3[2], fgfr3[3]
    dset = measurement_subgrp.create_dataset(target_names[3]+"->X", (len(fgfr4[0]),),
                                                      dtype=np.dtype([("41" , float),
                                                                    ("42", float),
                                                                    ("43", float),
                                                                    ("44", float)]))
    dset["41"], dset["42"], dset["43"], dset["44"] = fgfr4[0], fgfr4[1], fgfr4[2], fgfr4[3]

    
# distances below, threshold
dset = subgrp_one_below.create_dataset("threshold", (1,1), dtype=np.dtype([("threshold", int)]))
dset["threshold"] = threshold_neighbor_one_below
# distances below mean, not active
dset = subsubsubgrp_one_below_mean_nactive.create_dataset(target_names[0]+"->X", (1,),
                                                      dtype=np.dtype([("11" , float),
                                                                    ("\u0394 11", float),
                                                                    ("12", float),
                                                                    ("\u0394 12", float),
                                                                    ("13", float),
                                                                    ("\u0394 13", float),
                                                                    ("14", float),
                                                                    ("\u0394 14", float)]))
dset["11"] = distances_below_nactive_FGFR1X_mean[0]
dset["\u0394 11"] = distances_below_nactive_FGFR1X_mean_error[0]
dset["12"] = distances_below_nactive_FGFR1X_mean[1]
dset["\u0394 12"] = distances_below_nactive_FGFR1X_mean_error[1]
dset["13"] = distances_below_nactive_FGFR1X_mean[2]
dset["\u0394 13"] = distances_below_nactive_FGFR1X_mean_error[2]
dset["14"] = distances_below_nactive_FGFR1X_mean[3]
dset["\u0394 14"] = distances_below_nactive_FGFR1X_mean_error[3]
dset = subsubsubgrp_one_below_mean_nactive.create_dataset(target_names[1]+"->X", (1,),
                                                      dtype=np.dtype([("21" , float),
                                                                    ("\u0394 21", float),
                                                                    ("22", float),
                                                                    ("\u0394 22", float),
                                                                    ("23", float),
                                                                    ("\u0394 23", float),
                                                                    ("24", float),
                                                                    ("\u0394 24", float)]))
dset["21"] = distances_below_nactive_FGFR2X_mean[0]
dset["\u0394 21"] = distances_below_nactive_FGFR2X_mean_error[0]
dset["22"] = distances_below_nactive_FGFR2X_mean[1]
dset["\u0394 22"] = distances_below_nactive_FGFR2X_mean_error[1]
dset["23"] = distances_below_nactive_FGFR2X_mean[2]
dset["\u0394 23"] = distances_below_nactive_FGFR2X_mean_error[2]
dset["24"] = distances_below_nactive_FGFR2X_mean[3]
dset["\u0394 24"] = distances_below_nactive_FGFR2X_mean_error[3]
dset = subsubsubgrp_one_below_mean_nactive.create_dataset(target_names[2]+"->X", (1,),
                                                      dtype=np.dtype([("31" , float),
                                                                    ("\u0394 31", float),
                                                                    ("32", float),
                                                                    ("\u0394 32", float),
                                                                    ("33", float),
                                                                    ("\u0394 33", float),
                                                                    ("34", float),
                                                                    ("\u0394 34", float)]))
dset["31"] = distances_below_nactive_FGFR3X_mean[0]
dset["\u0394 31"] = distances_below_nactive_FGFR3X_mean_error[0]
dset["32"] = distances_below_nactive_FGFR3X_mean[1]
dset["\u0394 32"] = distances_below_nactive_FGFR3X_mean_error[1]
dset["33"] = distances_below_nactive_FGFR3X_mean[2]
dset["\u0394 33"] = distances_below_nactive_FGFR3X_mean_error[2]
dset["34"] = distances_below_nactive_FGFR3X_mean[3]
dset["\u0394 34"] = distances_below_nactive_FGFR3X_mean_error[3]
dset = subsubsubgrp_one_below_mean_nactive.create_dataset(target_names[3]+"->X", (1,),
                                                      dtype=np.dtype([("41" , float),
                                                                    ("\u0394 41", float),
                                                                    ("42", float),
                                                                    ("\u0394 42", float),
                                                                    ("43", float),
                                                                    ("\u0394 43", float),
                                                                    ("44", float),
                                                                    ("\u0394 44", float)]))
dset["41"] = distances_below_nactive_FGFR4X_mean[0]
dset["\u0394 41"] = distances_below_nactive_FGFR4X_mean_error[0]
dset["42"] = distances_below_nactive_FGFR4X_mean[1]
dset["\u0394 42"] = distances_below_nactive_FGFR4X_mean_error[1]
dset["43"] = distances_below_nactive_FGFR4X_mean[2]
dset["\u0394 43"] = distances_below_nactive_FGFR4X_mean_error[2]
dset["44"] = distances_below_nactive_FGFR4X_mean[3]
dset["\u0394 44"] = distances_below_nactive_FGFR4X_mean_error[3]
# distances below mean, active
dset = subsubsubgrp_one_below_mean_active.create_dataset(target_names[0]+"->X", (1,),
                                                      dtype=np.dtype([("11" , float),
                                                                    ("\u0394 11", float),
                                                                    ("12", float),
                                                                    ("\u0394 12", float),
                                                                    ("13", float),
                                                                    ("\u0394 13", float),
                                                                    ("14", float),
                                                                    ("\u0394 14", float)]))
dset["11"] = distances_below_active_FGFR1X_mean[0]
dset["\u0394 11"] = distances_below_active_FGFR1X_mean_error[0]
dset["12"] = distances_below_active_FGFR1X_mean[1]
dset["\u0394 12"] = distances_below_active_FGFR1X_mean_error[1]
dset["13"] = distances_below_active_FGFR1X_mean[2]
dset["\u0394 13"] = distances_below_active_FGFR1X_mean_error[2]
dset["14"] = distances_below_active_FGFR1X_mean[3]
dset["\u0394 14"] = distances_below_active_FGFR1X_mean_error[3]
dset = subsubsubgrp_one_below_mean_active.create_dataset(target_names[1]+"->X", (1,),
                                                      dtype=np.dtype([("21" , float),
                                                                    ("\u0394 21", float),
                                                                    ("22", float),
                                                                    ("\u0394 22", float),
                                                                    ("23", float),
                                                                    ("\u0394 23", float),
                                                                    ("24", float),
                                                                    ("\u0394 24", float)]))
dset["21"] = distances_below_active_FGFR2X_mean[0]
dset["\u0394 21"] = distances_below_active_FGFR2X_mean_error[0]
dset["22"] = distances_below_active_FGFR2X_mean[1]
dset["\u0394 22"] = distances_below_active_FGFR2X_mean_error[1]
dset["23"] = distances_below_active_FGFR2X_mean[2]
dset["\u0394 23"] = distances_below_active_FGFR2X_mean_error[2]
dset["24"] = distances_below_active_FGFR2X_mean[3]
dset["\u0394 24"] = distances_below_active_FGFR2X_mean_error[3]
dset = subsubsubgrp_one_below_mean_active.create_dataset(target_names[2]+"->X", (1,),
                                                      dtype=np.dtype([("31" , float),
                                                                    ("\u0394 31", float),
                                                                    ("32", float),
                                                                    ("\u0394 32", float),
                                                                    ("33", float),
                                                                    ("\u0394 33", float),
                                                                    ("34", float),
                                                                    ("\u0394 34", float)]))
dset["31"] = distances_below_active_FGFR3X_mean[0]
dset["\u0394 31"] = distances_below_active_FGFR3X_mean_error[0]
dset["32"] = distances_below_active_FGFR3X_mean[1]
dset["\u0394 32"] = distances_below_active_FGFR3X_mean_error[1]
dset["33"] = distances_below_active_FGFR3X_mean[2]
dset["\u0394 33"] = distances_below_active_FGFR3X_mean_error[2]
dset["34"] = distances_below_active_FGFR3X_mean[3]
dset["\u0394 34"] = distances_below_active_FGFR3X_mean_error[3]
dset = subsubsubgrp_one_below_mean_active.create_dataset(target_names[3]+"->X", (1,),
                                                      dtype=np.dtype([("41" , float),
                                                                    ("\u0394 41", float),
                                                                    ("42", float),
                                                                    ("\u0394 42", float),
                                                                    ("43", float),
                                                                    ("\u0394 43", float),
                                                                    ("44", float),
                                                                    ("\u0394 44", float)]))
dset["41"] = distances_below_active_FGFR4X_mean[0]
dset["\u0394 41"] = distances_below_active_FGFR4X_mean_error[0]
dset["42"] = distances_below_active_FGFR4X_mean[1]
dset["\u0394 42"] = distances_below_active_FGFR4X_mean_error[1]
dset["43"] = distances_below_active_FGFR4X_mean[2]
dset["\u0394 43"] = distances_below_active_FGFR4X_mean_error[2]
dset["44"] = distances_below_active_FGFR4X_mean[3]
dset["\u0394 44"] = distances_below_active_FGFR4X_mean_error[3]


# distances below single, not active
for i, (na_11, na_12, na_13, na_14, na_21, na_22, na_23, na_24, na_31, na_32, na_33, na_34, na_41, na_42, na_43, na_44) in enumerate(zip(FGFR11_distances_nactive_below, FGFR12_distances_nactive_below, FGFR13_distances_nactive_below, FGFR14_distances_nactive_below,
                                                                                                                                        FGFR21_distances_nactive_below, FGFR22_distances_nactive_below, FGFR23_distances_nactive_below, FGFR24_distances_nactive_below,
                                                                                                                                        FGFR31_distances_nactive_below, FGFR32_distances_nactive_below, FGFR33_distances_nactive_below, FGFR34_distances_nactive_below,
                                                                                                                                        FGFR41_distances_nactive_below, FGFR42_distances_nactive_below, FGFR43_distances_nactive_below, FGFR44_distances_nactive_below), 1):
    i = str(i) if i > 9 else "0" + str(i)
    measurement_subgrp = subsubsubgrp_one_below_single_nactive.create_group("measurement_" + i)
    dset = measurement_subgrp.create_dataset(target_names[0]+"->X", (1,),
                                                      dtype=np.dtype([("11" , float),
                                                                    ("12", float),
                                                                    ("13", float),
                                                                    ("14", float)]))
    dset["11"], dset["12"], dset["13"], dset["14"] = na_11[0], na_12[0], na_13[0], na_14[0]
    dset = measurement_subgrp.create_dataset(target_names[1]+"->X", (1,),
                                                      dtype=np.dtype([("21" , float),
                                                                    ("22", float),
                                                                    ("23", float),
                                                                    ("24", float)]))
    dset["21"], dset["22"], dset["23"], dset["24"] = na_21[0], na_22[0], na_23[0], na_24[0]
    dset = measurement_subgrp.create_dataset(target_names[2]+"->X", (1,),
                                                      dtype=np.dtype([("31" , float),
                                                                    ("32", float),
                                                                    ("33", float),
                                                                    ("34", float)]))
    dset["31"], dset["32"], dset["33"], dset["34"] = na_31[0], na_32[0], na_33[0], na_34[0]
    dset = measurement_subgrp.create_dataset(target_names[3]+"->X", (1,),
                                                      dtype=np.dtype([("41" , float),
                                                                    ("42", float),
                                                                    ("43", float),
                                                                    ("44", float)]))
    dset["41"], dset["42"], dset["43"], dset["44"] = na_41[0], na_42[0], na_43[0], na_44[0]

# distances below single, active
for i, (a_11, a_12, a_13, a_14, a_21, a_22, a_23, a_24, a_31, a_32, a_33, a_34, a_41, a_42, a_43, a_44) in enumerate(zip(FGFR11_distances_active_below, FGFR12_distances_active_below, FGFR13_distances_active_below, FGFR14_distances_active_below,
                                                                                                                                        FGFR21_distances_active_below, FGFR22_distances_active_below, FGFR23_distances_active_below, FGFR24_distances_active_below,
                                                                                                                                        FGFR31_distances_active_below, FGFR32_distances_active_below, FGFR33_distances_active_below, FGFR34_distances_active_below,
                                                                                                                                        FGFR41_distances_active_below, FGFR42_distances_active_below, FGFR43_distances_active_below, FGFR44_distances_active_below), 1):
    i = str(i) if i > 9 else "0" + str(i)
    measurement_subgrp = subsubsubgrp_one_below_single_active.create_group("measurement_" + i)
    dset = measurement_subgrp.create_dataset(target_names[0]+"->X", (1,),
                                                      dtype=np.dtype([("11" , float),
                                                                    ("12", float),
                                                                    ("13", float),
                                                                    ("14", float)]))
    dset["11"], dset["12"], dset["13"], dset["14"] = a_11[0], a_12[0], a_13[0], a_14[0]
    dset = measurement_subgrp.create_dataset(target_names[1]+"->X", (1,),
                                                      dtype=np.dtype([("21" , float),
                                                                    ("22", float),
                                                                    ("23", float),
                                                                    ("24", float)]))
    dset["21"], dset["22"], dset["23"], dset["24"] = a_21[0], a_22[0], a_23[0], a_24[0]
    dset = measurement_subgrp.create_dataset(target_names[2]+"->X", (1,),
                                                      dtype=np.dtype([("31" , float),
                                                                    ("32", float),
                                                                    ("33", float),
                                                                    ("34", float)]))
    dset["31"], dset["32"], dset["33"], dset["34"] = a_31[0], a_32[0], a_33[0], a_34[0]
    dset = measurement_subgrp.create_dataset(target_names[3]+"->X", (1,),
                                                      dtype=np.dtype([("41" , float),
                                                                    ("42", float),
                                                                    ("43", float),
                                                                    ("44", float)]))
    dset["41"], dset["42"], dset["43"], dset["44"] = a_41[0], a_42[0], a_43[0], a_44[0]

    
# distances all neighbors mean, bin size
dset = subsubgrp_distances_all_mean.create_dataset("bin_size", (1,1), dtype=np.dtype([("bin size", int)]))
dset["bin size"] = bin_size_neighbor_all
# distances all neighbors mean, not active
dset = subsubgrp_distances_all_mean.create_dataset("mean "+condition_names[0], (len(FGFR1X_nactive_hist_mean),),
                                                      dtype=np.dtype([("distance", float),
                                                                    (target_names[0] + "X", float),
                                                                    ("\u0394 " + target_names[0] + "X", float),
                                                                    (target_names[1] + "X", float),
                                                                    ("\u0394 " + target_names[1] + "X", float),
                                                                    (target_names[2] + "X", float),
                                                                    ("\u0394 " + target_names[2] + "X", float),
                                                                    (target_names[3] + "X", float),
                                                                    ("\u0394 " + target_names[3] + "X", float)]))

dset["distance"] = bins_plot_neighbor_all[:-1]
dset[target_names[0] + "X"] = FGFR1X_nactive_hist_mean
dset["\u0394 " + target_names[0] + "X"] = FGFR1X_nactive_hist_mean_error
dset[target_names[1] + "X"] = FGFR2X_nactive_hist_mean
dset["\u0394 " + target_names[1] + "X"] = FGFR2X_nactive_hist_mean_error
dset[target_names[2] + "X"] = FGFR3X_nactive_hist_mean
dset["\u0394 " + target_names[2] + "X"] = FGFR3X_nactive_hist_mean_error
dset[target_names[3] + "X"] = FGFR4X_nactive_hist_mean
dset["\u0394 " + target_names[3] + "X"] = FGFR4X_nactive_hist_mean_error
# distances all neighbors mean, active
dset = subsubgrp_distances_all_mean.create_dataset("mean "+condition_names[1], (len(FGFR1X_active_hist_mean),),
                                                      dtype=np.dtype([("distance", float),
                                                                    (target_names[0] + "X", float),
                                                                    ("\u0394 " + target_names[0] + "X", float),
                                                                    (target_names[1] + "X", float),
                                                                    ("\u0394 " + target_names[1] + "X", float),
                                                                    (target_names[2] + "X", float),
                                                                    ("\u0394 " + target_names[2] + "X", float),
                                                                    (target_names[3] + "X", float),
                                                                    ("\u0394 " + target_names[3] + "X", float)]))
dset["distance"] = bins_plot_neighbor_all[:-1]
dset[target_names[0] + "X"] = FGFR1X_active_hist_mean
dset["\u0394 " + target_names[0] + "X"] = FGFR1X_active_hist_mean_error
dset[target_names[1] + "X"] = FGFR2X_active_hist_mean
dset["\u0394 " + target_names[1] + "X"] = FGFR2X_active_hist_mean_error
dset[target_names[2] + "X"] = FGFR3X_active_hist_mean
dset["\u0394 " + target_names[2] + "X"] = FGFR3X_active_hist_mean_error
dset[target_names[3] + "X"] = FGFR4X_active_hist_mean
dset["\u0394 " + target_names[3] + "X"] = FGFR4X_active_hist_mean_error
# distances all neighbors single, not active
for i, (fgfr1, fgfr2, fgfr3, fgfr4) in enumerate(zip(FGFR1X_distances_nactive, FGFR2X_distances_nactive, FGFR3X_distances_nactive, FGFR4X_distances_nactive), 1):
    i = str(i) if i > 9 else "0" + str(i)
    measurement_subgrp = subsubsubgrp_distances_all_single_nactive.create_group("measurement_" + i)
    
    dset = measurement_subgrp.create_dataset(target_names[0]+"->X", (len(fgfr1),),
                                                      dtype=np.dtype([(target_names[0]+"X", float)]))
    dset[target_names[0]+"X"] = fgfr1
    dset = measurement_subgrp.create_dataset(target_names[1]+"->X", (len(fgfr2),),
                                                      dtype=np.dtype([(target_names[1]+"X", float)]))
    dset[target_names[1]+"X"] = fgfr2
    dset = measurement_subgrp.create_dataset(target_names[2]+"->X", (len(fgfr3),),
                                                      dtype=np.dtype([(target_names[2]+"X", float)]))
    dset[target_names[2]+"X"] = fgfr3
    dset = measurement_subgrp.create_dataset(target_names[3]+"->X", (len(fgfr4),),
                                                      dtype=np.dtype([(target_names[3]+"X", float)]))
    dset[target_names[3]+"X"] = fgfr4
# distances all neighbors single, active
for i, (fgfr1, fgfr2, fgfr3, fgfr4) in enumerate(zip(FGFR1X_distances_active, FGFR2X_distances_active, FGFR3X_distances_active, FGFR4X_distances_active), 1):
    i = str(i) if i > 9 else "0" + str(i)
    measurement_subgrp = subsubsubgrp_distances_all_single_active.create_group("measurement_" + i)
    
    dset = measurement_subgrp.create_dataset(target_names[0]+"->X", (len(fgfr1),),
                                                      dtype=np.dtype([(target_names[0]+"X", float)]))
    dset[target_names[0]+"X"] = fgfr1
    dset = measurement_subgrp.create_dataset(target_names[1]+"->X", (len(fgfr2),),
                                                      dtype=np.dtype([(target_names[1]+"X", float)]))
    dset[target_names[1]+"X"] = fgfr2
    dset = measurement_subgrp.create_dataset(target_names[2]+"->X", (len(fgfr3),),
                                                      dtype=np.dtype([(target_names[2]+"X", float)]))
    dset[target_names[2]+"X"] = fgfr3
    dset = measurement_subgrp.create_dataset(target_names[3]+"->X", (len(fgfr4),),
                                                      dtype=np.dtype([(target_names[3]+"X", float)]))
    dset[target_names[3]+"X"] = fgfr4


# types all neighbors mean, not active
dset = subsubsubgrp_types_all_mean_nactive.create_dataset(target_names[0]+"->X", (1,1), dtype=np.dtype([("11", float),
                                                                                   ("\u0394 11", float),
                                                                                   ("12", float),
                                                                                   ("\u0394 12", float),
                                                                                   ("13", float),
                                                                                   ("\u0394 13", float),
                                                                                   ("14", float),
                                                                                   ("\u0394 14", float)]))
dset["11"] = FGFR1X_types_nactive_counts_mean[0]
dset["\u0394 11"] = FGFR1X_types_nactive_counts_mean_error[0]
dset["12"] = FGFR1X_types_nactive_counts_mean[1]
dset["\u0394 12"] = FGFR1X_types_nactive_counts_mean_error[1]
dset["13"] = FGFR1X_types_nactive_counts_mean[2]
dset["\u0394 13"] = FGFR1X_types_nactive_counts_mean_error[2]
dset["14"] = FGFR1X_types_nactive_counts_mean[3]
dset["\u0394 14"] = FGFR1X_types_nactive_counts_mean_error[3]
dset = subsubsubgrp_types_all_mean_nactive.create_dataset(target_names[1]+"->X", (1,1), dtype=np.dtype([("21", float),
                                                                                   ("\u0394 21", float),
                                                                                   ("22", float),
                                                                                   ("\u0394 22", float),
                                                                                   ("23", float),
                                                                                   ("\u0394 23", float),
                                                                                   ("24", float),
                                                                                   ("\u0394 24", float)]))
dset["21"] = FGFR2X_types_nactive_counts_mean[0]
dset["\u0394 21"] = FGFR2X_types_nactive_counts_mean_error[0]
dset["22"] = FGFR2X_types_nactive_counts_mean[1]
dset["\u0394 22"] = FGFR2X_types_nactive_counts_mean_error[1]
dset["23"] = FGFR2X_types_nactive_counts_mean[2]
dset["\u0394 23"] = FGFR2X_types_nactive_counts_mean_error[2]
dset["24"] = FGFR2X_types_nactive_counts_mean[3]
dset["\u0394 24"] = FGFR2X_types_nactive_counts_mean_error[3]
dset = subsubsubgrp_types_all_mean_nactive.create_dataset(target_names[2]+"->X", (1,1), dtype=np.dtype([("31", float),
                                                                                   ("\u0394 31", float),
                                                                                   ("32", float),
                                                                                   ("\u0394 32", float),
                                                                                   ("33", float),
                                                                                   ("\u0394 33", float),
                                                                                   ("34", float),
                                                                                   ("\u0394 34", float)]))
dset["31"] = FGFR3X_types_nactive_counts_mean[0]
dset["\u0394 31"] = FGFR3X_types_nactive_counts_mean_error[0]
dset["32"] = FGFR3X_types_nactive_counts_mean[1]
dset["\u0394 32"] = FGFR3X_types_nactive_counts_mean_error[1]
dset["33"] = FGFR3X_types_nactive_counts_mean[2]
dset["\u0394 33"] = FGFR3X_types_nactive_counts_mean_error[2]
dset["34"] = FGFR3X_types_nactive_counts_mean[3]
dset["\u0394 34"] = FGFR3X_types_nactive_counts_mean_error[3]
dset = subsubsubgrp_types_all_mean_nactive.create_dataset(target_names[3]+"->X", (1,1), dtype=np.dtype([("41", float),
                                                                                   ("\u0394 41", float),
                                                                                   ("42", float),
                                                                                   ("\u0394 42", float),
                                                                                   ("43", float),
                                                                                   ("\u0394 43", float),
                                                                                   ("44", float),
                                                                                   ("\u0394 44", float)]))
dset["41"] = FGFR4X_types_nactive_counts_mean[0]
dset["\u0394 41"] = FGFR4X_types_nactive_counts_mean_error[0]
dset["42"] = FGFR4X_types_nactive_counts_mean[1]
dset["\u0394 42"] = FGFR4X_types_nactive_counts_mean_error[1]
dset["43"] = FGFR4X_types_nactive_counts_mean[2]
dset["\u0394 43"] = FGFR4X_types_nactive_counts_mean_error[2]
dset["44"] = FGFR4X_types_nactive_counts_mean[3]
dset["\u0394 44"] = FGFR4X_types_nactive_counts_mean_error[3]
# types all neighbors mean, active
dset = subsubsubgrp_types_all_mean_active.create_dataset(target_names[0]+"->X", (1,1), dtype=np.dtype([("11", float),
                                                                                   ("\u0394 11", float),
                                                                                   ("12", float),
                                                                                   ("\u0394 12", float),
                                                                                   ("13", float),
                                                                                   ("\u0394 13", float),
                                                                                   ("14", float),
                                                                                   ("\u0394 14", float)]))
dset["11"] = FGFR1X_types_active_counts_mean[0]
dset["\u0394 11"] = FGFR1X_types_active_counts_mean_error[0]
dset["12"] = FGFR1X_types_active_counts_mean[1]
dset["\u0394 12"] = FGFR1X_types_active_counts_mean_error[1]
dset["13"] = FGFR1X_types_active_counts_mean[2]
dset["\u0394 13"] = FGFR1X_types_active_counts_mean_error[2]
dset["14"] = FGFR1X_types_active_counts_mean[3]
dset["\u0394 14"] = FGFR1X_types_active_counts_mean_error[3]
dset = subsubsubgrp_types_all_mean_active.create_dataset(target_names[1]+"->X", (1,1), dtype=np.dtype([("21", float),
                                                                                   ("\u0394 21", float),
                                                                                   ("22", float),
                                                                                   ("\u0394 22", float),
                                                                                   ("23", float),
                                                                                   ("\u0394 23", float),
                                                                                   ("24", float),
                                                                                   ("\u0394 24", float)]))
dset["21"] = FGFR2X_types_active_counts_mean[0]
dset["\u0394 21"] = FGFR2X_types_active_counts_mean_error[0]
dset["22"] = FGFR2X_types_active_counts_mean[1]
dset["\u0394 22"] = FGFR2X_types_active_counts_mean_error[1]
dset["23"] = FGFR2X_types_active_counts_mean[2]
dset["\u0394 23"] = FGFR2X_types_active_counts_mean_error[2]
dset["24"] = FGFR2X_types_active_counts_mean[3]
dset["\u0394 24"] = FGFR2X_types_active_counts_mean_error[3]
dset = subsubsubgrp_types_all_mean_active.create_dataset(target_names[2]+"->X", (1,1), dtype=np.dtype([("31", float),
                                                                                   ("\u0394 31", float),
                                                                                   ("32", float),
                                                                                   ("\u0394 32", float),
                                                                                   ("33", float),
                                                                                   ("\u0394 33", float),
                                                                                   ("34", float),
                                                                                   ("\u0394 34", float)]))
dset["31"] = FGFR3X_types_active_counts_mean[0]
dset["\u0394 31"] = FGFR3X_types_active_counts_mean_error[0]
dset["32"] = FGFR3X_types_active_counts_mean[1]
dset["\u0394 32"] = FGFR3X_types_active_counts_mean_error[1]
dset["33"] = FGFR3X_types_active_counts_mean[2]
dset["\u0394 33"] = FGFR3X_types_active_counts_mean_error[2]
dset["34"] = FGFR3X_types_active_counts_mean[3]
dset["\u0394 34"] = FGFR3X_types_active_counts_mean_error[3]
dset = subsubsubgrp_types_all_mean_active.create_dataset(target_names[3]+"->X", (1,1), dtype=np.dtype([("41", float),
                                                                                   ("\u0394 41", float),
                                                                                   ("42", float),
                                                                                   ("\u0394 42", float),
                                                                                   ("43", float),
                                                                                   ("\u0394 43", float),
                                                                                   ("44", float),
                                                                                   ("\u0394 44", float)]))
dset["41"] = FGFR4X_types_active_counts_mean[0]
dset["\u0394 41"] = FGFR4X_types_active_counts_mean_error[0]
dset["42"] = FGFR4X_types_active_counts_mean[1]
dset["\u0394 42"] = FGFR4X_types_active_counts_mean_error[1]
dset["43"] = FGFR4X_types_active_counts_mean[2]
dset["\u0394 43"] = FGFR4X_types_active_counts_mean_error[2]
dset["44"] = FGFR4X_types_active_counts_mean[3]
dset["\u0394 44"] = FGFR4X_types_active_counts_mean_error[3]

# types all neighbors, single not active
for i, (na_1x, na_2x, na_3x, na_4x) in enumerate(zip(FGFR1X_types_nactive_counts, FGFR2X_types_nactive_counts, FGFR3X_types_nactive_counts, FGFR4X_types_nactive_counts), 1):
    i = str(i) if i > 9 else "0" + str(i)
    measurement_subgrp = subsubsubgrp_types_all_single_nactive.create_group("measurement_" + i)
    dset = measurement_subgrp.create_dataset(target_names[0]+"->X", (1,1), dtype=np.dtype([("11", float),
                                                                                   ("12", float),
                                                                                   ("13", float),
                                                                                   ("14", float)]))
    dset["11"], dset["12"], dset["13"], dset["14"] = na_1x[0], na_1x[1], na_1x[2], na_1x[3]
    dset = measurement_subgrp.create_dataset(target_names[1]+"->X", (1,1), dtype=np.dtype([("21", float),
                                                                                   ("22", float),
                                                                                   ("23", float),
                                                                                   ("24", float)]))
    dset["21"], dset["22"], dset["23"], dset["24"] = na_2x[0], na_2x[1], na_2x[2], na_2x[3]
    dset = measurement_subgrp.create_dataset(target_names[2]+"->X", (1,1), dtype=np.dtype([("31", float),
                                                                                   ("32", float),
                                                                                   ("33", float),
                                                                                   ("34", float)]))
    dset["31"], dset["32"], dset["33"], dset["34"] = na_3x[0], na_3x[1], na_3x[2], na_3x[3]
    dset = measurement_subgrp.create_dataset(target_names[3]+"->X", (1,1), dtype=np.dtype([("41", float),
                                                                                   ("42", float),
                                                                                   ("43", float),
                                                                                   ("44", float)]))
    dset["41"], dset["42"], dset["43"], dset["44"] = na_4x[0], na_4x[1], na_4x[2], na_4x[3]
# types all neighbors, single active
for i, (a_1x, a_2x, a_3x, a_4x) in enumerate(zip(FGFR1X_types_active_counts, FGFR2X_types_active_counts, FGFR3X_types_active_counts, FGFR4X_types_active_counts), 1):
    i = str(i) if i > 9 else "0" + str(i)
    measurement_subgrp = subsubsubgrp_types_all_single_active.create_group("measurement_" + i)
    dset = measurement_subgrp.create_dataset(target_names[0]+"->X", (1,1), dtype=np.dtype([("11", float),
                                                                                   ("12", float),
                                                                                   ("13", float),
                                                                                   ("14", float)]))
    dset["11"], dset["12"], dset["13"], dset["14"] = a_1x[0], a_1x[1], a_1x[2], a_1x[3]
    dset = measurement_subgrp.create_dataset(target_names[1]+"->X", (1,1), dtype=np.dtype([("21", float),
                                                                                   ("22", float),
                                                                                   ("23", float),
                                                                                   ("24", float)]))
    dset["21"], dset["22"], dset["23"], dset["24"] = a_2x[0], a_2x[1], a_2x[2], a_2x[3]
    dset = measurement_subgrp.create_dataset(target_names[2]+"->X", (1,1), dtype=np.dtype([("31", float),
                                                                                   ("32", float),
                                                                                   ("33", float),
                                                                                   ("34", float)]))
    dset["31"], dset["32"], dset["33"], dset["34"] = a_3x[0], a_3x[1], a_3x[2], a_3x[3]
    dset = measurement_subgrp.create_dataset(target_names[3]+"->X", (1,1), dtype=np.dtype([("41", float),
                                                                                   ("42", float),
                                                                                   ("43", float),
                                                                                   ("44", float)]))
    dset["41"], dset["42"], dset["43"], dset["44"] = a_4x[0], a_4x[1], a_4x[2], a_4x[3]


# distances all below, threshold
dset = subgrp_all_below.create_dataset("threshold", (1,1), dtype=np.dtype([("threshold", int)]))
dset["threshold"] = threshold_all_distances_below
# distances all below, mean not active
dset = subsubgrp_all_below_mean.create_dataset("mean "+condition_names[0], (1,1),
                                                      dtype=np.dtype([(target_names[0] + "X", float),
                                                                    ("\u0394 " + target_names[0] + "X", float),
                                                                    (target_names[1] + "X", float),
                                                                    ("\u0394 " + target_names[1] + "X", float),
                                                                    (target_names[2] + "X", float),
                                                                    ("\u0394 " + target_names[2] + "X", float),
                                                                    (target_names[3] + "X", float),
                                                                     ("\u0394 " + target_names[3] + "X", float)]))
dset[target_names[0] + "X"] = FGFR_distances_nactive_mean[0]
dset["\u0394 " + target_names[0] + "X"] = FGFR_distances_nactive_mean_error[0]
dset[target_names[1] + "X"] = FGFR_distances_nactive_mean[1]
dset["\u0394 " + target_names[1] + "X"] = FGFR_distances_nactive_mean_error[1]
dset[target_names[2] + "X"] = FGFR_distances_nactive_mean[2]
dset["\u0394 " + target_names[2] + "X"] = FGFR_distances_nactive_mean_error[2]
dset[target_names[3] + "X"] = FGFR_distances_nactive_mean[3]
dset["\u0394 " + target_names[3] + "X"] = FGFR_distances_nactive_mean_error[3]
# distances all below, mean active
dset = subsubgrp_all_below_mean.create_dataset("mean "+condition_names[1], (1,1),
                                                      dtype=np.dtype([(target_names[0] + "X", float),
                                                                    ("\u0394 " + target_names[0] + "X", float),
                                                                    (target_names[1] + "X", float),
                                                                    ("\u0394 " + target_names[1] + "X", float),
                                                                    (target_names[2] + "X", float),
                                                                    ("\u0394 " + target_names[2] + "X", float),
                                                                    (target_names[3] + "X", float),
                                                                     ("\u0394 " + target_names[3] + "X", float)]))
dset[target_names[0] + "X"] = FGFR_distances_active_mean[0]
dset["\u0394 " + target_names[0] + "X"] = FGFR_distances_active_mean_error[0]
dset[target_names[1] + "X"] = FGFR_distances_active_mean[1]
dset["\u0394 " + target_names[1] + "X"] = FGFR_distances_active_mean_error[1]
dset[target_names[2] + "X"] = FGFR_distances_active_mean[2]
dset["\u0394 " + target_names[2] + "X"] = FGFR_distances_active_mean_error[2]
dset[target_names[3] + "X"] = FGFR_distances_active_mean[3]
dset["\u0394 " + target_names[3] + "X"] = FGFR_distances_active_mean_error[3]

# distances below single, not active
for i, (fgfr1, fgfr2, fgfr3, fgfr4) in enumerate(zip(FGFR1X_distances_nactive_below, FGFR2X_distances_nactive_below, FGFR3X_distances_nactive_below, FGFR4X_distances_nactive_below), 1):
    i = str(i) if i > 9 else "0" + str(i)
    dset = subsubsubgrp_all_below_single_nactive.create_dataset("measurement_" + i, (1,1),
                                                      dtype=np.dtype([(target_names[0] + "X", float),
                                                                    (target_names[1] + "X", float),
                                                                    (target_names[2] + "X", float),
                                                                    (target_names[3] + "X", float)]))
    dset[target_names[0] + "X"], dset[target_names[1] + "X"], dset[target_names[2] + "X"], dset[target_names[3] + "X"] = fgfr1, fgfr2, fgfr3, fgfr4

# distances below single, active
for i, (fgfr1, fgfr2, fgfr3, fgfr4) in enumerate(zip(FGFR1X_distances_active_below, FGFR2X_distances_active_below, FGFR3X_distances_active_below, FGFR4X_distances_active_below), 1):
    i = str(i) if i > 9 else "0" + str(i)
    dset = subsubsubgrp_all_below_single_active.create_dataset("measurement_" + i, (1,1),
                                                      dtype=np.dtype([(target_names[0] + "X", float),
                                                                    (target_names[1] + "X", float),
                                                                    (target_names[2] + "X", float),
                                                                    (target_names[3] + "X", float)]))
    dset[target_names[0] + "X"], dset[target_names[1] + "X"], dset[target_names[2] + "X"], dset[target_names[3] + "X"] = fgfr1, fgfr2, fgfr3, fgfr4

h5.close()